In [ ]:
## This file implements neural networks and logistic regression on p0040presabsSTCC_quant.
## For fully-connected neural networks, the accuracy is 75.32%, and 76.62%% after improvement.
## We further implement the over-sampling method and the combination of over- and under-sampling method, in case of imbalanced data.
## For fully-connected neural networks, the accuracy is 86.44% after regularization for combination data
## and 75.76% for over-sampling data.
## To improve neural network performance, we implement lasso regression to select important features.
## With the new dataset, the accuracy becomes 77.92% without balanced methods, 72.88% for combination, and 80.81% for over-sampling data.
## For logistic regression, the accuracies are 76.62% without balanced methods, 94.92 for combination data and 85.86% for over-sampling data.
## For random forest, the accuracy is 79.22% without balanced methods, 100% for combination, and 86.87% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 8257% without balanced methods, 91.93% for combination data, and 83.09% for over-sampling.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0040presabsSTCC_quant.csv')
df.shape

(255, 375)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0.814000
1      0.365125
2      0.803875
3      1.088125
4      1.081750
5      1.112875
6      0.817875
7      0.398250
8      0.569500
9      0.526125
10     0.351625
11     0.814875
12     0.590875
13     0.373875
14     0.922625
15     0.955750
16     0.383500
17     0.379625
18     0.689625
19     0.186750
20     0.351750
21     0.364250
22     0.306125
23     1.112250
24     0.963000
25     0.159500
26     0.824750
27     0.196000
28     0.590875
29     0.830375
         ...   
225    0.715500
226    0.469000
227    0.670400
228    0.746000
229    0.969000
230    0.577125
231    0.627375
232    0.930500
233    0.478875
234    0.602500
235    0.900125
236    0.646875
237    0.847125
238    0.461500
239    0.258375
240    0.515875
241    0.466875
242    0.275500
243    0.542125
244    0.486625
245    0.765375
246    0.845625
247    0.530875
248    0.894625
249    0.457250
250    0.868000
251    0.550500
252    0.685125
253    0.939500
254    0.926500
Name: pheno, Length: 255

In [5]:
df.head()

,id,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,...,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno,ST,CC
0,107,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0.814000,5,5
1,109,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0.365125,8,8
2,115,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.803875,5,5
3,120335,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1.088125,5,5
4,120337,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1.081750,5,5


In [6]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [7]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [8]:
df['pheno']
df['pheno'].value_counts()

1    165
0     90
Name: pheno, dtype: int64

In [9]:
df.shape

(255, 375)

In [10]:
df_clean = df.drop(columns=['id'])

In [11]:
df_clean.shape

(255, 374)

In [12]:
df_clean.head()

,TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT,TTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGT,TTTTACAAAAGAGGAAGCAGATAAACTTTATCAACCTATCGGTTCTTCGCAGCCGTCACTGAATATTTGGACAGGCAGTGAAACAGAATATAATTATTTG,TTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTTAGTAGAGTG,TTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTC,TTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATCTTTTGTTGTGTCAGTACTTACAACTTT,TTTGGACAGGCAGTGAAAC,TTTGGACAGGCAGTGAAACAGAATATAATTATTTGTA,TTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGTTTTAGTTTGATC,TTTCGCTGTTGCAACATCTTTAACAGGTGTTTGAACTTTATTTTGTTCTTGAGCAGTTTGTGCTGTTTTAACTGTATGAGCAGTTTGTGCTGTTTTAACT,...,group_10124,group_1648,group_426,group_8122,group_8213,group_8913,group_9905,pheno,ST,CC
0,1,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,5,5
1,0,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,8,8
2,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
3,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
4,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5


In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 373) (255,)


In [14]:
# split into train, test data (original)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

Using TensorFlow backend.


In [17]:
#### neural network
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [18]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/100
178/178 [==============================] - 0s 892us/step - loss: 13.7483 - accuracy: 0.5169 - val_loss: 0.6851 - val_accuracy: 0.6104
Epoch 2/100
178/178 [==============================] - 0s 101us/step - loss: 0.6833 - accuracy: 0.6124 - val_loss: 0.6802 - val_accuracy: 0.6234
Epoch 3/100
178/178 [==============================] - 0s 131us/step - loss: 0.6613 - accuracy: 0.6573 - val_loss: 0.6683 - val_accuracy: 0.6104
Epoch 4/100
178/178 [==============================] - 0s 118us/step - loss: 0.6431 - accuracy: 0.6629 - val_loss: 0.6830 - val_accuracy: 0.5974
Epoch 5/100
178/178 [==============================] - 0s 124us/step - loss: 0.6314 - accuracy: 0.6798 - val_loss: 0.6752 - val_accuracy: 0.5844
Epoch 6/100
178/178 [==============================] - 0s 133us/step - loss: 0.6083 - accuracy: 0.6910 - val_loss: 0.6569 - val_accuracy: 0.6104
Epoch 7/100
178/178 [==============================] - 0s 141us/step - loss: 0.6084 

Epoch 57/100
178/178 [==============================] - 0s 149us/step - loss: 0.4858 - accuracy: 0.8202 - val_loss: 0.6844 - val_accuracy: 0.7532
Epoch 58/100
178/178 [==============================] - 0s 169us/step - loss: 0.5038 - accuracy: 0.7921 - val_loss: 0.6708 - val_accuracy: 0.7662
Epoch 59/100
178/178 [==============================] - 0s 135us/step - loss: 0.4849 - accuracy: 0.8090 - val_loss: 0.6883 - val_accuracy: 0.7403
Epoch 60/100
178/178 [==============================] - 0s 125us/step - loss: 0.4880 - accuracy: 0.8090 - val_loss: 0.6910 - val_accuracy: 0.7403
Epoch 61/100
178/178 [==============================] - 0s 115us/step - loss: 0.4866 - accuracy: 0.8034 - val_loss: 0.7198 - val_accuracy: 0.7013
Epoch 62/100
178/178 [==============================] - 0s 126us/step - loss: 0.4907 - accuracy: 0.7978 - val_loss: 0.6784 - val_accuracy: 0.7662
Epoch 63/100
178/178 [==============================] - 0s 134us/step - loss: 0.4755 - accuracy: 0.8146 - val_loss: 0.6878 -

In [20]:
acc_test1 = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test1*100))

77/77 [==============================] - 0s 114us/step
test accuracy: 75.32%


In [37]:
#### improve neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [38]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
model2.fit(X_train, y_train,
          batch_size=16, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 694us/step - loss: 5.9805 - accuracy: 0.5955 - val_loss: 6.5692 - val_accuracy: 0.3766
Epoch 2/1000
178/178 [==============================] - 0s 195us/step - loss: 9.1498 - accuracy: 0.5169 - val_loss: 0.6851 - val_accuracy: 0.6494
Epoch 3/1000
178/178 [==============================] - 0s 212us/step - loss: 0.6777 - accuracy: 0.6404 - val_loss: 0.6771 - val_accuracy: 0.6364
Epoch 4/1000
178/178 [==============================] - 0s 211us/step - loss: 0.5948 - accuracy: 0.6517 - val_loss: 0.6819 - val_accuracy: 0.6623
Epoch 5/1000
178/178 [==============================] - 0s 194us/step - loss: 0.6083 - accuracy: 0.6629 - val_loss: 0.6791 - val_accuracy: 0.6623
Epoch 6/1000
178/178 [==============================] - 0s 207us/step - loss: 0.5889 - accuracy: 0.7022 - val_loss: 0.6795 - val_accuracy: 0.6494
Epoch 7/1000
178/178 [==============================] - 0s 206us/step - loss: 0

178/178 [==============================] - 0s 201us/step - loss: 0.4483 - accuracy: 0.8090 - val_loss: 0.6529 - val_accuracy: 0.7662
Epoch 57/1000
178/178 [==============================] - 0s 215us/step - loss: 0.4413 - accuracy: 0.8202 - val_loss: 0.6519 - val_accuracy: 0.7532
Epoch 58/1000
178/178 [==============================] - 0s 173us/step - loss: 0.4274 - accuracy: 0.8146 - val_loss: 0.6425 - val_accuracy: 0.7403
Epoch 59/1000
178/178 [==============================] - 0s 157us/step - loss: 0.4203 - accuracy: 0.8202 - val_loss: 0.7016 - val_accuracy: 0.7403
Epoch 60/1000
178/178 [==============================] - 0s 174us/step - loss: 0.4223 - accuracy: 0.8202 - val_loss: 0.7287 - val_accuracy: 0.7532
Epoch 61/1000
178/178 [==============================] - 0s 167us/step - loss: 0.4219 - accuracy: 0.8202 - val_loss: 0.7067 - val_accuracy: 0.7273
Epoch 62/1000
178/178 [==============================] - 0s 170us/step - loss: 0.4612 - accuracy: 0.8146 - val_loss: 0.6287 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 209us/step - loss: 0.3823 - accuracy: 0.8202 - val_loss: 0.6806 - val_accuracy: 0.7403
Epoch 113/1000
178/178 [==============================] - 0s 179us/step - loss: 0.3962 - accuracy: 0.8371 - val_loss: 0.6255 - val_accuracy: 0.7792
Epoch 114/1000
178/178 [==============================] - 0s 164us/step - loss: 0.3890 - accuracy: 0.8371 - val_loss: 0.6536 - val_accuracy: 0.7273
Epoch 115/1000
178/178 [==============================] - 0s 165us/step - loss: 0.3853 - accuracy: 0.8202 - val_loss: 0.6618 - val_accuracy: 0.7273
Epoch 116/1000
178/178 [==============================] - 0s 169us/step - loss: 0.3838 - accuracy: 0.8258 - val_loss: 0.6636 - val_accuracy: 0.7273
Epoch 117/1000
178/178 [==============================] - 0s 172us/step - loss: 0.3858 - accuracy: 0.8371 - val_loss: 0.6551 - val_accuracy: 0.7403
Epoch 118/1000
178/178 [==============================] - 0s 177us/step - loss: 0.3895 - accuracy: 0.8315 - val_

178/178 [==============================] - 0s 168us/step - loss: 0.3909 - accuracy: 0.8258 - val_loss: 0.7110 - val_accuracy: 0.7403
Epoch 168/1000
178/178 [==============================] - 0s 172us/step - loss: 0.3723 - accuracy: 0.8371 - val_loss: 0.7168 - val_accuracy: 0.7273
Epoch 169/1000
178/178 [==============================] - 0s 173us/step - loss: 0.3569 - accuracy: 0.8371 - val_loss: 0.7026 - val_accuracy: 0.7403
Epoch 170/1000
178/178 [==============================] - 0s 156us/step - loss: 0.3508 - accuracy: 0.8427 - val_loss: 0.7366 - val_accuracy: 0.7403
Epoch 171/1000
178/178 [==============================] - 0s 156us/step - loss: 0.3451 - accuracy: 0.8427 - val_loss: 0.7448 - val_accuracy: 0.7403
Epoch 172/1000
178/178 [==============================] - 0s 182us/step - loss: 0.3578 - accuracy: 0.8258 - val_loss: 0.7150 - val_accuracy: 0.7403
Epoch 173/1000
178/178 [==============================] - 0s 184us/step - loss: 0.3425 - accuracy: 0.8427 - val_loss: 0.7972 - 

178/178 [==============================] - 0s 179us/step - loss: 0.3483 - accuracy: 0.8315 - val_loss: 0.7496 - val_accuracy: 0.7273
Epoch 223/1000
178/178 [==============================] - 0s 179us/step - loss: 0.3584 - accuracy: 0.8427 - val_loss: 0.7363 - val_accuracy: 0.7662
Epoch 224/1000
178/178 [==============================] - 0s 173us/step - loss: 0.3245 - accuracy: 0.8539 - val_loss: 0.7631 - val_accuracy: 0.7273
Epoch 225/1000
178/178 [==============================] - 0s 184us/step - loss: 0.3268 - accuracy: 0.8483 - val_loss: 0.7364 - val_accuracy: 0.7532
Epoch 226/1000
178/178 [==============================] - 0s 175us/step - loss: 0.3231 - accuracy: 0.8539 - val_loss: 0.8866 - val_accuracy: 0.7403
Epoch 227/1000
178/178 [==============================] - 0s 166us/step - loss: 0.3291 - accuracy: 0.8427 - val_loss: 0.7565 - val_accuracy: 0.7273
Epoch 228/1000
178/178 [==============================] - 0s 173us/step - loss: 0.3243 - accuracy: 0.8427 - val_loss: 0.8113 - 

...

178/178 [==============================] - 0s 166us/step - loss: 0.2867 - accuracy: 0.8708 - val_loss: 1.4372 - val_accuracy: 0.7922
Epoch 773/1000
178/178 [==============================] - 0s 181us/step - loss: 0.2864 - accuracy: 0.8708 - val_loss: 1.4475 - val_accuracy: 0.7662
Epoch 774/1000
178/178 [==============================] - 0s 175us/step - loss: 0.2866 - accuracy: 0.8708 - val_loss: 1.4131 - val_accuracy: 0.7662
Epoch 775/1000
178/178 [==============================] - 0s 165us/step - loss: 0.2867 - accuracy: 0.8708 - val_loss: 1.4448 - val_accuracy: 0.7662
Epoch 776/1000
178/178 [==============================] - 0s 166us/step - loss: 0.2864 - accuracy: 0.8708 - val_loss: 1.4400 - val_accuracy: 0.7922
Epoch 777/1000
178/178 [==============================] - 0s 165us/step - loss: 0.2862 - accuracy: 0.8708 - val_loss: 1.4294 - val_accuracy: 0.7792
Epoch 778/1000
178/178 [==============================] - 0s 167us/step - loss: 0.2862 - accuracy: 0.8708 - val_loss: 1.4492 - 

178/178 [==============================] - 0s 161us/step - loss: 0.2857 - accuracy: 0.8708 - val_loss: 1.4746 - val_accuracy: 0.7792
Epoch 828/1000
178/178 [==============================] - 0s 164us/step - loss: 0.2854 - accuracy: 0.8708 - val_loss: 1.4717 - val_accuracy: 0.7792
Epoch 829/1000
178/178 [==============================] - 0s 153us/step - loss: 0.2854 - accuracy: 0.8708 - val_loss: 1.5011 - val_accuracy: 0.7662
Epoch 830/1000
178/178 [==============================] - 0s 172us/step - loss: 0.2855 - accuracy: 0.8708 - val_loss: 1.4971 - val_accuracy: 0.7922
Epoch 831/1000
178/178 [==============================] - 0s 192us/step - loss: 0.2854 - accuracy: 0.8708 - val_loss: 1.4907 - val_accuracy: 0.7922
Epoch 832/1000
178/178 [==============================] - 0s 166us/step - loss: 0.2855 - accuracy: 0.8708 - val_loss: 1.5021 - val_accuracy: 0.7792
Epoch 833/1000
178/178 [==============================] - 0s 165us/step - loss: 0.2851 - accuracy: 0.8708 - val_loss: 1.4953 - 

178/178 [==============================] - 0s 164us/step - loss: 0.2846 - accuracy: 0.8708 - val_loss: 1.5338 - val_accuracy: 0.7792
Epoch 883/1000
178/178 [==============================] - 0s 176us/step - loss: 0.2847 - accuracy: 0.8708 - val_loss: 1.5183 - val_accuracy: 0.7792
Epoch 884/1000
178/178 [==============================] - 0s 170us/step - loss: 0.2848 - accuracy: 0.8708 - val_loss: 1.5279 - val_accuracy: 0.7922
Epoch 885/1000
178/178 [==============================] - 0s 162us/step - loss: 0.2845 - accuracy: 0.8708 - val_loss: 1.5350 - val_accuracy: 0.7922
Epoch 886/1000
178/178 [==============================] - 0s 150us/step - loss: 0.2845 - accuracy: 0.8708 - val_loss: 1.5263 - val_accuracy: 0.7922
Epoch 887/1000
178/178 [==============================] - 0s 168us/step - loss: 0.2847 - accuracy: 0.8708 - val_loss: 1.5270 - val_accuracy: 0.7922
Epoch 888/1000
178/178 [==============================] - 0s 169us/step - loss: 0.2845 - accuracy: 0.8708 - val_loss: 1.5186 - 

178/178 [==============================] - 0s 168us/step - loss: 0.2841 - accuracy: 0.8708 - val_loss: 1.5615 - val_accuracy: 0.7922
Epoch 938/1000
178/178 [==============================] - 0s 178us/step - loss: 0.2841 - accuracy: 0.8708 - val_loss: 1.5650 - val_accuracy: 0.7922
Epoch 939/1000
178/178 [==============================] - 0s 175us/step - loss: 0.2839 - accuracy: 0.8708 - val_loss: 1.5542 - val_accuracy: 0.7792
Epoch 940/1000
178/178 [==============================] - 0s 193us/step - loss: 0.2841 - accuracy: 0.8708 - val_loss: 1.5561 - val_accuracy: 0.7792
Epoch 941/1000
178/178 [==============================] - 0s 177us/step - loss: 0.2842 - accuracy: 0.8708 - val_loss: 1.5557 - val_accuracy: 0.7792
Epoch 942/1000
178/178 [==============================] - 0s 175us/step - loss: 0.2839 - accuracy: 0.8708 - val_loss: 1.5594 - val_accuracy: 0.7922
Epoch 943/1000
178/178 [==============================] - 0s 175us/step - loss: 0.2841 - accuracy: 0.8708 - val_loss: 1.5548 - 

178/178 [==============================] - 0s 174us/step - loss: 0.3536 - accuracy: 0.8539 - val_loss: 1.5702 - val_accuracy: 0.7792
Epoch 993/1000
178/178 [==============================] - 0s 176us/step - loss: 0.2884 - accuracy: 0.8708 - val_loss: 1.5529 - val_accuracy: 0.7792
Epoch 994/1000
178/178 [==============================] - 0s 172us/step - loss: 0.2879 - accuracy: 0.8708 - val_loss: 1.5683 - val_accuracy: 0.7792
Epoch 995/1000
178/178 [==============================] - 0s 160us/step - loss: 0.2875 - accuracy: 0.8708 - val_loss: 1.5595 - val_accuracy: 0.7792
Epoch 996/1000
178/178 [==============================] - 0s 166us/step - loss: 0.2872 - accuracy: 0.8708 - val_loss: 1.5574 - val_accuracy: 0.7792
Epoch 997/1000
178/178 [==============================] - 0s 156us/step - loss: 0.2871 - accuracy: 0.8708 - val_loss: 1.5506 - val_accuracy: 0.7662
Epoch 998/1000
178/178 [==============================] - 0s 165us/step - loss: 0.2870 - accuracy: 0.8708 - val_loss: 1.5645 - 

In [40]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 128us/step
test accuracy: 76.62%


In [41]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 105), (1, 91)]


In [42]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 165), (1, 165)]


In [43]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [44]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [45]:
############# Fully-Connected Neural Network ################

In [46]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [47]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [48]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [49]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 137 samples, validate on 59 samples
Epoch 1/1000
137/137 [==============================] - 0s 741us/step - loss: 2.6907 - accuracy: 0.4088 - val_loss: 1.0175 - val_accuracy: 0.5424
Epoch 2/1000
137/137 [==============================] - 0s 107us/step - loss: 0.8004 - accuracy: 0.6350 - val_loss: 0.5812 - val_accuracy: 0.7119
Epoch 3/1000
137/137 [==============================] - 0s 133us/step - loss: 0.5971 - accuracy: 0.7080 - val_loss: 0.5667 - val_accuracy: 0.7458
Epoch 4/1000
137/137 [==============================] - 0s 149us/step - loss: 0.5624 - accuracy: 0.7956 - val_loss: 0.5369 - val_accuracy: 0.7627
Epoch 5/1000
137/137 [==============================] - 0s 148us/step - loss: 0.5337 - accuracy: 0.8102 - val_loss: 0.5218 - val_accuracy: 0.7627
Epoch 6/1000
137/137 [==============================] - 0s 147us/step - loss: 0.5151 - accuracy: 0.8321 - val_loss: 0.5077 - val_accuracy: 0.7627
Epoch 7/1000
137/137 [==============================] - 0s 161us/step - loss: 0

137/137 [==============================] - 0s 211us/step - loss: 0.3096 - accuracy: 0.8467 - val_loss: 0.3850 - val_accuracy: 0.7797
Epoch 57/1000
137/137 [==============================] - 0s 153us/step - loss: 0.3132 - accuracy: 0.8467 - val_loss: 0.3854 - val_accuracy: 0.7797
Epoch 58/1000
137/137 [==============================] - 0s 143us/step - loss: 0.3074 - accuracy: 0.8467 - val_loss: 0.3784 - val_accuracy: 0.7627
Epoch 59/1000
137/137 [==============================] - 0s 124us/step - loss: 0.3038 - accuracy: 0.8321 - val_loss: 0.3789 - val_accuracy: 0.7797
Epoch 60/1000
137/137 [==============================] - 0s 119us/step - loss: 0.3046 - accuracy: 0.8467 - val_loss: 0.3789 - val_accuracy: 0.7627
Epoch 61/1000
137/137 [==============================] - 0s 110us/step - loss: 0.3080 - accuracy: 0.8467 - val_loss: 0.3836 - val_accuracy: 0.7627
Epoch 62/1000
137/137 [==============================] - 0s 132us/step - loss: 0.3021 - accuracy: 0.8467 - val_loss: 0.3748 - val_ac

Epoch 112/1000
137/137 [==============================] - 0s 105us/step - loss: 0.2802 - accuracy: 0.8540 - val_loss: 0.3460 - val_accuracy: 0.7966
Epoch 113/1000
137/137 [==============================] - 0s 110us/step - loss: 0.2858 - accuracy: 0.7956 - val_loss: 0.3442 - val_accuracy: 0.8136
Epoch 114/1000
137/137 [==============================] - 0s 139us/step - loss: 0.2797 - accuracy: 0.8394 - val_loss: 0.3512 - val_accuracy: 0.7966
Epoch 115/1000
137/137 [==============================] - 0s 141us/step - loss: 0.2756 - accuracy: 0.8540 - val_loss: 0.3414 - val_accuracy: 0.7797
Epoch 116/1000
137/137 [==============================] - ETA: 0s - loss: 0.5223 - accuracy: 0.71 - 0s 132us/step - loss: 0.3781 - accuracy: 0.8102 - val_loss: 0.4631 - val_accuracy: 0.7797
Epoch 117/1000
137/137 [==============================] - 0s 114us/step - loss: 0.2611 - accuracy: 0.8686 - val_loss: 0.4432 - val_accuracy: 0.7627
Epoch 118/1000
137/137 [==============================] - 0s 112us/ste

137/137 [==============================] - 0s 122us/step - loss: 0.2753 - accuracy: 0.8540 - val_loss: 0.3191 - val_accuracy: 0.8305
Epoch 168/1000
137/137 [==============================] - 0s 135us/step - loss: 0.2602 - accuracy: 0.8613 - val_loss: 0.3182 - val_accuracy: 0.8136
Epoch 169/1000
137/137 [==============================] - 0s 122us/step - loss: 0.2543 - accuracy: 0.8759 - val_loss: 0.3675 - val_accuracy: 0.7797
Epoch 170/1000
137/137 [==============================] - 0s 122us/step - loss: 0.2611 - accuracy: 0.8613 - val_loss: 0.3516 - val_accuracy: 0.7966
Epoch 171/1000
137/137 [==============================] - 0s 116us/step - loss: 0.2701 - accuracy: 0.8613 - val_loss: 0.3224 - val_accuracy: 0.7966
Epoch 172/1000
137/137 [==============================] - 0s 182us/step - loss: 0.2576 - accuracy: 0.8540 - val_loss: 0.3186 - val_accuracy: 0.8136
Epoch 173/1000
137/137 [==============================] - 0s 128us/step - loss: 0.2599 - accuracy: 0.8832 - val_loss: 0.3173 - 

137/137 [==============================] - 0s 144us/step - loss: 0.2591 - accuracy: 0.8540 - val_loss: 0.3161 - val_accuracy: 0.8136
Epoch 223/1000
137/137 [==============================] - 0s 166us/step - loss: 0.2602 - accuracy: 0.8613 - val_loss: 0.3668 - val_accuracy: 0.7627
Epoch 224/1000
137/137 [==============================] - 0s 200us/step - loss: 0.2637 - accuracy: 0.8686 - val_loss: 0.3277 - val_accuracy: 0.7966
Epoch 225/1000
137/137 [==============================] - 0s 190us/step - loss: 0.2595 - accuracy: 0.8686 - val_loss: 0.2959 - val_accuracy: 0.8136
Epoch 226/1000
137/137 [==============================] - 0s 135us/step - loss: 0.2557 - accuracy: 0.8686 - val_loss: 0.2962 - val_accuracy: 0.7966
Epoch 227/1000
137/137 [==============================] - 0s 176us/step - loss: 0.2561 - accuracy: 0.8540 - val_loss: 0.2971 - val_accuracy: 0.7966
Epoch 228/1000
137/137 [==============================] - 0s 133us/step - loss: 0.2585 - accuracy: 0.8540 - val_loss: 0.2957 - 

...

137/137 [==============================] - 0s 104us/step - loss: 0.2082 - accuracy: 0.8832 - val_loss: 0.2234 - val_accuracy: 0.8644
Epoch 773/1000
137/137 [==============================] - 0s 115us/step - loss: 0.2083 - accuracy: 0.8832 - val_loss: 0.2239 - val_accuracy: 0.8644
Epoch 774/1000
137/137 [==============================] - 0s 124us/step - loss: 0.2083 - accuracy: 0.8832 - val_loss: 0.2250 - val_accuracy: 0.8644
Epoch 775/1000
137/137 [==============================] - 0s 128us/step - loss: 0.2081 - accuracy: 0.8832 - val_loss: 0.2248 - val_accuracy: 0.8644
Epoch 776/1000
137/137 [==============================] - 0s 135us/step - loss: 0.2083 - accuracy: 0.8832 - val_loss: 0.2233 - val_accuracy: 0.8644
Epoch 777/1000
137/137 [==============================] - 0s 126us/step - loss: 0.2083 - accuracy: 0.8832 - val_loss: 0.2221 - val_accuracy: 0.8644
Epoch 778/1000
137/137 [==============================] - 0s 122us/step - loss: 0.2084 - accuracy: 0.8832 - val_loss: 0.2215 - 

137/137 [==============================] - 0s 115us/step - loss: 0.2071 - accuracy: 0.8832 - val_loss: 0.2200 - val_accuracy: 0.8644
Epoch 828/1000
137/137 [==============================] - 0s 110us/step - loss: 0.2069 - accuracy: 0.8832 - val_loss: 0.2198 - val_accuracy: 0.8644
Epoch 829/1000
137/137 [==============================] - 0s 129us/step - loss: 0.2069 - accuracy: 0.8832 - val_loss: 0.2189 - val_accuracy: 0.8644
Epoch 830/1000
137/137 [==============================] - 0s 115us/step - loss: 0.2070 - accuracy: 0.8832 - val_loss: 0.2197 - val_accuracy: 0.8644
Epoch 831/1000
137/137 [==============================] - 0s 122us/step - loss: 0.2067 - accuracy: 0.8832 - val_loss: 0.2198 - val_accuracy: 0.8644
Epoch 832/1000
137/137 [==============================] - 0s 124us/step - loss: 0.2067 - accuracy: 0.8832 - val_loss: 0.2190 - val_accuracy: 0.8644
Epoch 833/1000
137/137 [==============================] - 0s 152us/step - loss: 0.2068 - accuracy: 0.8832 - val_loss: 0.2195 - 

137/137 [==============================] - 0s 99us/step - loss: 0.2061 - accuracy: 0.8832 - val_loss: 0.2254 - val_accuracy: 0.8644
Epoch 883/1000
137/137 [==============================] - 0s 113us/step - loss: 0.2061 - accuracy: 0.8832 - val_loss: 0.2232 - val_accuracy: 0.8644
Epoch 884/1000
137/137 [==============================] - 0s 120us/step - loss: 0.2062 - accuracy: 0.8832 - val_loss: 0.2230 - val_accuracy: 0.8644
Epoch 885/1000
137/137 [==============================] - 0s 122us/step - loss: 0.2062 - accuracy: 0.8832 - val_loss: 0.2327 - val_accuracy: 0.8644
Epoch 886/1000
137/137 [==============================] - 0s 141us/step - loss: 0.2062 - accuracy: 0.8832 - val_loss: 0.2261 - val_accuracy: 0.8644
Epoch 887/1000
137/137 [==============================] - 0s 138us/step - loss: 0.2059 - accuracy: 0.8832 - val_loss: 0.2223 - val_accuracy: 0.8644
Epoch 888/1000
137/137 [==============================] - 0s 123us/step - loss: 0.2060 - accuracy: 0.8832 - val_loss: 0.2213 - v

137/137 [==============================] - 0s 98us/step - loss: 0.2050 - accuracy: 0.8832 - val_loss: 0.2213 - val_accuracy: 0.8644
Epoch 938/1000
137/137 [==============================] - 0s 100us/step - loss: 0.2050 - accuracy: 0.8832 - val_loss: 0.2167 - val_accuracy: 0.8644
Epoch 939/1000
137/137 [==============================] - 0s 126us/step - loss: 0.2049 - accuracy: 0.8832 - val_loss: 0.2175 - val_accuracy: 0.8644
Epoch 940/1000
137/137 [==============================] - 0s 107us/step - loss: 0.2049 - accuracy: 0.8832 - val_loss: 0.2186 - val_accuracy: 0.8644
Epoch 941/1000
137/137 [==============================] - 0s 109us/step - loss: 0.2049 - accuracy: 0.8832 - val_loss: 0.2190 - val_accuracy: 0.8644
Epoch 942/1000
137/137 [==============================] - 0s 111us/step - loss: 0.2049 - accuracy: 0.8832 - val_loss: 0.2181 - val_accuracy: 0.8644
Epoch 943/1000
137/137 [==============================] - 0s 116us/step - loss: 0.2050 - accuracy: 0.8832 - val_loss: 0.2185 - v

137/137 [==============================] - 0s 117us/step - loss: 0.2042 - accuracy: 0.8832 - val_loss: 0.2183 - val_accuracy: 0.8644
Epoch 993/1000
137/137 [==============================] - 0s 129us/step - loss: 0.2042 - accuracy: 0.8832 - val_loss: 0.2170 - val_accuracy: 0.8644
Epoch 994/1000
137/137 [==============================] - 0s 134us/step - loss: 0.2043 - accuracy: 0.8832 - val_loss: 0.2160 - val_accuracy: 0.8644
Epoch 995/1000
137/137 [==============================] - 0s 116us/step - loss: 0.2042 - accuracy: 0.8832 - val_loss: 0.2148 - val_accuracy: 0.8644
Epoch 996/1000
137/137 [==============================] - 0s 136us/step - loss: 0.2043 - accuracy: 0.8832 - val_loss: 0.2159 - val_accuracy: 0.8644
Epoch 997/1000
137/137 [==============================] - 0s 149us/step - loss: 0.2042 - accuracy: 0.8832 - val_loss: 0.2162 - val_accuracy: 0.8644
Epoch 998/1000
137/137 [==============================] - 0s 125us/step - loss: 0.2042 - accuracy: 0.8832 - val_loss: 0.2164 - 

In [50]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

59/59 [==============================] - 0s 107us/step
combination test accuracy: 86.44%


In [51]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [52]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [53]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 137 samples, validate on 59 samples
Epoch 1/1000
137/137 [==============================] - 0s 1ms/step - loss: 4.1664 - accuracy: 0.6204 - val_loss: 1.0981 - val_accuracy: 0.7797
Epoch 2/1000
137/137 [==============================] - 0s 108us/step - loss: 2.7643 - accuracy: 0.7007 - val_loss: 0.7801 - val_accuracy: 0.7797
Epoch 3/1000
137/137 [==============================] - 0s 138us/step - loss: 2.1157 - accuracy: 0.6934 - val_loss: 0.7390 - val_accuracy: 0.7797
Epoch 4/1000
137/137 [==============================] - 0s 151us/step - loss: 2.2564 - accuracy: 0.7372 - val_loss: 0.7097 - val_accuracy: 0.6949
Epoch 5/1000
137/137 [==============================] - 0s 154us/step - loss: 1.6714 - accuracy: 0.7956 - val_loss: 0.6716 - val_accuracy: 0.7288
Epoch 6/1000
137/137 [==============================] - 0s 158us/step - loss: 1.7445 - accuracy: 0.7883 - val_loss: 0.6398 - val_accuracy: 0.7797
Epoch 7/1000
137/137 [==============================] - 0s 153us/step - loss: 2.2

137/137 [==============================] - 0s 165us/step - loss: 1.2583 - accuracy: 0.8394 - val_loss: 0.5141 - val_accuracy: 0.7966
Epoch 57/1000
137/137 [==============================] - 0s 148us/step - loss: 2.3975 - accuracy: 0.7372 - val_loss: 0.5138 - val_accuracy: 0.7966
Epoch 58/1000
137/137 [==============================] - 0s 181us/step - loss: 1.8413 - accuracy: 0.7737 - val_loss: 0.5138 - val_accuracy: 0.7966
Epoch 59/1000
137/137 [==============================] - 0s 156us/step - loss: 1.8246 - accuracy: 0.7810 - val_loss: 0.5119 - val_accuracy: 0.7966
Epoch 60/1000
137/137 [==============================] - 0s 148us/step - loss: 1.9191 - accuracy: 0.7956 - val_loss: 0.5123 - val_accuracy: 0.7966
Epoch 61/1000
137/137 [==============================] - 0s 135us/step - loss: 1.9421 - accuracy: 0.7664 - val_loss: 0.5075 - val_accuracy: 0.7966
Epoch 62/1000
137/137 [==============================] - 0s 150us/step - loss: 2.2752 - accuracy: 0.7445 - val_loss: 0.5082 - val_ac

Epoch 112/1000
137/137 [==============================] - 0s 129us/step - loss: 2.2238 - accuracy: 0.7664 - val_loss: 0.4751 - val_accuracy: 0.7627
Epoch 113/1000
137/137 [==============================] - 0s 144us/step - loss: 1.6691 - accuracy: 0.7883 - val_loss: 0.4662 - val_accuracy: 0.7966
Epoch 114/1000
137/137 [==============================] - 0s 133us/step - loss: 1.7849 - accuracy: 0.7737 - val_loss: 0.4608 - val_accuracy: 0.8136
Epoch 115/1000
137/137 [==============================] - 0s 147us/step - loss: 2.3264 - accuracy: 0.7518 - val_loss: 0.4661 - val_accuracy: 0.7966
Epoch 116/1000
137/137 [==============================] - 0s 194us/step - loss: 1.3490 - accuracy: 0.8175 - val_loss: 0.4908 - val_accuracy: 0.7627
Epoch 117/1000
137/137 [==============================] - 0s 158us/step - loss: 2.1210 - accuracy: 0.7518 - val_loss: 0.4729 - val_accuracy: 0.7797
Epoch 118/1000
137/137 [==============================] - 0s 161us/step - loss: 1.7795 - accuracy: 0.7883 - val_

137/137 [==============================] - 0s 125us/step - loss: 1.6601 - accuracy: 0.7956 - val_loss: 0.4540 - val_accuracy: 0.7966
Epoch 168/1000
137/137 [==============================] - 0s 140us/step - loss: 1.6666 - accuracy: 0.7883 - val_loss: 0.4470 - val_accuracy: 0.7797
Epoch 169/1000
137/137 [==============================] - 0s 200us/step - loss: 1.6521 - accuracy: 0.7956 - val_loss: 0.4378 - val_accuracy: 0.8136
Epoch 170/1000
137/137 [==============================] - 0s 162us/step - loss: 1.7787 - accuracy: 0.7810 - val_loss: 0.4485 - val_accuracy: 0.7797
Epoch 171/1000
137/137 [==============================] - 0s 150us/step - loss: 1.4272 - accuracy: 0.8102 - val_loss: 0.4447 - val_accuracy: 0.7797
Epoch 172/1000
137/137 [==============================] - 0s 144us/step - loss: 1.4352 - accuracy: 0.8029 - val_loss: 0.4445 - val_accuracy: 0.7797
Epoch 173/1000
137/137 [==============================] - 0s 164us/step - loss: 1.9942 - accuracy: 0.7664 - val_loss: 0.4433 - 

137/137 [==============================] - 0s 146us/step - loss: 1.9872 - accuracy: 0.7591 - val_loss: 0.4211 - val_accuracy: 0.8136
Epoch 223/1000
137/137 [==============================] - 0s 128us/step - loss: 1.9828 - accuracy: 0.7664 - val_loss: 0.4158 - val_accuracy: 0.8305
Epoch 224/1000
137/137 [==============================] - 0s 150us/step - loss: 1.6579 - accuracy: 0.7810 - val_loss: 0.4207 - val_accuracy: 0.8136
Epoch 225/1000
137/137 [==============================] - 0s 134us/step - loss: 1.2984 - accuracy: 0.8248 - val_loss: 0.4176 - val_accuracy: 0.8136
Epoch 226/1000
137/137 [==============================] - 0s 135us/step - loss: 1.8639 - accuracy: 0.7883 - val_loss: 0.4625 - val_accuracy: 0.7288
Epoch 227/1000
137/137 [==============================] - 0s 165us/step - loss: 1.6475 - accuracy: 0.7956 - val_loss: 0.4253 - val_accuracy: 0.7966
Epoch 228/1000
137/137 [==============================] - 0s 140us/step - loss: 1.5341 - accuracy: 0.8029 - val_loss: 0.4268 - 

...

137/137 [==============================] - 0s 145us/step - loss: 1.8003 - accuracy: 0.8029 - val_loss: 0.3436 - val_accuracy: 0.8305
Epoch 773/1000
137/137 [==============================] - 0s 130us/step - loss: 1.3593 - accuracy: 0.8248 - val_loss: 0.3427 - val_accuracy: 0.8305
Epoch 774/1000
137/137 [==============================] - 0s 141us/step - loss: 1.4445 - accuracy: 0.8321 - val_loss: 0.3379 - val_accuracy: 0.8305
Epoch 775/1000
137/137 [==============================] - 0s 140us/step - loss: 2.1342 - accuracy: 0.7737 - val_loss: 0.3465 - val_accuracy: 0.8305
Epoch 776/1000
137/137 [==============================] - 0s 130us/step - loss: 2.3671 - accuracy: 0.7518 - val_loss: 0.3355 - val_accuracy: 0.8305
Epoch 777/1000
137/137 [==============================] - 0s 148us/step - loss: 1.8077 - accuracy: 0.7956 - val_loss: 0.3477 - val_accuracy: 0.8305
Epoch 778/1000
137/137 [==============================] - 0s 153us/step - loss: 1.6047 - accuracy: 0.7810 - val_loss: 0.3374 - 

137/137 [==============================] - 0s 158us/step - loss: 1.3473 - accuracy: 0.8321 - val_loss: 0.3349 - val_accuracy: 0.8305
Epoch 828/1000
137/137 [==============================] - 0s 114us/step - loss: 1.3735 - accuracy: 0.8175 - val_loss: 0.3380 - val_accuracy: 0.8305
Epoch 829/1000
137/137 [==============================] - 0s 137us/step - loss: 1.2664 - accuracy: 0.8102 - val_loss: 0.3385 - val_accuracy: 0.8305
Epoch 830/1000
137/137 [==============================] - 0s 137us/step - loss: 1.5709 - accuracy: 0.8175 - val_loss: 0.3391 - val_accuracy: 0.8305
Epoch 831/1000
137/137 [==============================] - 0s 141us/step - loss: 1.9275 - accuracy: 0.7737 - val_loss: 0.3313 - val_accuracy: 0.8305
Epoch 832/1000
137/137 [==============================] - 0s 125us/step - loss: 1.9306 - accuracy: 0.7737 - val_loss: 0.3447 - val_accuracy: 0.8305
Epoch 833/1000
137/137 [==============================] - 0s 146us/step - loss: 1.6801 - accuracy: 0.8102 - val_loss: 0.3376 - 

137/137 [==============================] - 0s 145us/step - loss: 1.1379 - accuracy: 0.8394 - val_loss: 0.3350 - val_accuracy: 0.8644
Epoch 883/1000
137/137 [==============================] - 0s 129us/step - loss: 1.5485 - accuracy: 0.8321 - val_loss: 0.3302 - val_accuracy: 0.8644
Epoch 884/1000
137/137 [==============================] - 0s 138us/step - loss: 1.8042 - accuracy: 0.7956 - val_loss: 0.3345 - val_accuracy: 0.8644
Epoch 885/1000
137/137 [==============================] - 0s 138us/step - loss: 2.1533 - accuracy: 0.7518 - val_loss: 0.3317 - val_accuracy: 0.8644
Epoch 886/1000
137/137 [==============================] - 0s 133us/step - loss: 2.5471 - accuracy: 0.7810 - val_loss: 0.3272 - val_accuracy: 0.8475
Epoch 887/1000
137/137 [==============================] - 0s 139us/step - loss: 1.2395 - accuracy: 0.8321 - val_loss: 0.3359 - val_accuracy: 0.8644
Epoch 888/1000
137/137 [==============================] - 0s 133us/step - loss: 1.7102 - accuracy: 0.7810 - val_loss: 0.3342 - 

137/137 [==============================] - 0s 126us/step - loss: 1.3509 - accuracy: 0.8321 - val_loss: 0.3437 - val_accuracy: 0.8475
Epoch 938/1000
137/137 [==============================] - 0s 126us/step - loss: 1.5845 - accuracy: 0.8029 - val_loss: 0.3294 - val_accuracy: 0.8475
Epoch 939/1000
137/137 [==============================] - 0s 129us/step - loss: 1.5774 - accuracy: 0.8248 - val_loss: 0.3412 - val_accuracy: 0.8644
Epoch 940/1000
137/137 [==============================] - 0s 137us/step - loss: 2.1457 - accuracy: 0.7664 - val_loss: 0.3315 - val_accuracy: 0.8644
Epoch 941/1000
137/137 [==============================] - 0s 137us/step - loss: 1.3436 - accuracy: 0.8321 - val_loss: 0.3343 - val_accuracy: 0.8644
Epoch 942/1000
137/137 [==============================] - 0s 142us/step - loss: 1.7922 - accuracy: 0.8102 - val_loss: 0.4095 - val_accuracy: 0.7288
Epoch 943/1000
137/137 [==============================] - 0s 134us/step - loss: 2.0522 - accuracy: 0.7810 - val_loss: 0.3930 - 

137/137 [==============================] - 0s 127us/step - loss: 1.2627 - accuracy: 0.8175 - val_loss: 0.3309 - val_accuracy: 0.8644
Epoch 993/1000
137/137 [==============================] - 0s 142us/step - loss: 2.3358 - accuracy: 0.7591 - val_loss: 0.3288 - val_accuracy: 0.8644
Epoch 994/1000
137/137 [==============================] - 0s 177us/step - loss: 1.5786 - accuracy: 0.8102 - val_loss: 0.3272 - val_accuracy: 0.8644
Epoch 995/1000
137/137 [==============================] - 0s 142us/step - loss: 2.0163 - accuracy: 0.7883 - val_loss: 0.3266 - val_accuracy: 0.8644
Epoch 996/1000
137/137 [==============================] - 0s 156us/step - loss: 1.8058 - accuracy: 0.7883 - val_loss: 0.3267 - val_accuracy: 0.8644
Epoch 997/1000
137/137 [==============================] - 0s 137us/step - loss: 2.2424 - accuracy: 0.7664 - val_loss: 0.3273 - val_accuracy: 0.8644
Epoch 998/1000
137/137 [==============================] - 0s 147us/step - loss: 1.4624 - accuracy: 0.8175 - val_loss: 0.3290 - 

In [54]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

59/59 [==============================] - 0s 107us/step
combination test accuracy: 86.44%


In [55]:
#### improve neural network
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [56]:
model1_comb3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [57]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 137 samples, validate on 59 samples
Epoch 1/1000
137/137 [==============================] - 0s 1ms/step - loss: 3.8919 - accuracy: 0.4234 - val_loss: 1.1723 - val_accuracy: 0.5424
Epoch 2/1000
137/137 [==============================] - 0s 95us/step - loss: 2.0274 - accuracy: 0.5328 - val_loss: 0.8422 - val_accuracy: 0.5593
Epoch 3/1000
137/137 [==============================] - 0s 152us/step - loss: 1.9147 - accuracy: 0.6496 - val_loss: 0.7923 - val_accuracy: 0.6441
Epoch 4/1000
137/137 [==============================] - 0s 156us/step - loss: 2.2933 - accuracy: 0.6350 - val_loss: 0.7514 - val_accuracy: 0.7119
Epoch 5/1000
137/137 [==============================] - 0s 157us/step - loss: 1.4295 - accuracy: 0.6861 - val_loss: 0.7304 - val_accuracy: 0.6271
Epoch 6/1000
137/137 [==============================] - 0s 171us/step - loss: 1.9017 - accuracy: 0.6934 - val_loss: 0.7140 - val_accuracy: 0.6271
Epoch 7/1000
137/137 [==============================] - 0s 169us/step - loss: 2.01

137/137 [==============================] - 0s 140us/step - loss: 1.9470 - accuracy: 0.7810 - val_loss: 0.5373 - val_accuracy: 0.7627
Epoch 57/1000
137/137 [==============================] - 0s 130us/step - loss: 1.6050 - accuracy: 0.8029 - val_loss: 0.5376 - val_accuracy: 0.7458
Epoch 58/1000
137/137 [==============================] - 0s 125us/step - loss: 1.7309 - accuracy: 0.7737 - val_loss: 0.5353 - val_accuracy: 0.7458
Epoch 59/1000
137/137 [==============================] - 0s 137us/step - loss: 2.1481 - accuracy: 0.7664 - val_loss: 0.5353 - val_accuracy: 0.7288
Epoch 60/1000
137/137 [==============================] - 0s 128us/step - loss: 1.6092 - accuracy: 0.7883 - val_loss: 0.5332 - val_accuracy: 0.7458
Epoch 61/1000
137/137 [==============================] - 0s 116us/step - loss: 1.7993 - accuracy: 0.7883 - val_loss: 0.5300 - val_accuracy: 0.7458
Epoch 62/1000
137/137 [==============================] - 0s 126us/step - loss: 1.6858 - accuracy: 0.8102 - val_loss: 0.5297 - val_ac

Epoch 112/1000
137/137 [==============================] - 0s 105us/step - loss: 1.1961 - accuracy: 0.8467 - val_loss: 0.4727 - val_accuracy: 0.8136
Epoch 113/1000
137/137 [==============================] - 0s 110us/step - loss: 2.0041 - accuracy: 0.7664 - val_loss: 0.4725 - val_accuracy: 0.8136
Epoch 114/1000
137/137 [==============================] - 0s 136us/step - loss: 1.7751 - accuracy: 0.7883 - val_loss: 0.4714 - val_accuracy: 0.8136
Epoch 115/1000
137/137 [==============================] - 0s 149us/step - loss: 1.4352 - accuracy: 0.8102 - val_loss: 0.4708 - val_accuracy: 0.8305
Epoch 116/1000
137/137 [==============================] - 0s 143us/step - loss: 1.9186 - accuracy: 0.7591 - val_loss: 0.4701 - val_accuracy: 0.8475
Epoch 117/1000
137/137 [==============================] - 0s 139us/step - loss: 1.6491 - accuracy: 0.8029 - val_loss: 0.4768 - val_accuracy: 0.8136
Epoch 118/1000
137/137 [==============================] - 0s 142us/step - loss: 1.3395 - accuracy: 0.8248 - val_

137/137 [==============================] - 0s 127us/step - loss: 2.0757 - accuracy: 0.7810 - val_loss: 0.4418 - val_accuracy: 0.8475
Epoch 168/1000
137/137 [==============================] - 0s 137us/step - loss: 1.8845 - accuracy: 0.7810 - val_loss: 0.4406 - val_accuracy: 0.8475
Epoch 169/1000
137/137 [==============================] - 0s 141us/step - loss: 1.3357 - accuracy: 0.8029 - val_loss: 0.4403 - val_accuracy: 0.8475
Epoch 170/1000
137/137 [==============================] - 0s 139us/step - loss: 1.6623 - accuracy: 0.7883 - val_loss: 0.4393 - val_accuracy: 0.8475
Epoch 171/1000
137/137 [==============================] - 0s 135us/step - loss: 1.6415 - accuracy: 0.8102 - val_loss: 0.4390 - val_accuracy: 0.8475
Epoch 172/1000
137/137 [==============================] - 0s 134us/step - loss: 1.8645 - accuracy: 0.7883 - val_loss: 0.4380 - val_accuracy: 0.8475
Epoch 173/1000
137/137 [==============================] - 0s 129us/step - loss: 1.3181 - accuracy: 0.8175 - val_loss: 0.4381 - 

137/137 [==============================] - 0s 171us/step - loss: 1.7219 - accuracy: 0.8175 - val_loss: 0.4216 - val_accuracy: 0.8475
Epoch 223/1000
137/137 [==============================] - 0s 126us/step - loss: 1.2937 - accuracy: 0.8321 - val_loss: 0.4187 - val_accuracy: 0.8475
Epoch 224/1000
137/137 [==============================] - 0s 139us/step - loss: 1.4998 - accuracy: 0.8321 - val_loss: 0.4203 - val_accuracy: 0.8475
Epoch 225/1000
137/137 [==============================] - 0s 135us/step - loss: 1.9823 - accuracy: 0.7664 - val_loss: 0.4190 - val_accuracy: 0.8475
Epoch 226/1000
137/137 [==============================] - 0s 144us/step - loss: 1.6430 - accuracy: 0.7956 - val_loss: 0.4202 - val_accuracy: 0.8475
Epoch 227/1000
137/137 [==============================] - 0s 131us/step - loss: 2.4044 - accuracy: 0.7518 - val_loss: 0.4235 - val_accuracy: 0.8475
Epoch 228/1000
137/137 [==============================] - 0s 137us/step - loss: 1.9708 - accuracy: 0.7737 - val_loss: 0.4183 - 

...

137/137 [==============================] - 0s 132us/step - loss: 1.7864 - accuracy: 0.8175 - val_loss: 0.3708 - val_accuracy: 0.8475
Epoch 773/1000
137/137 [==============================] - 0s 139us/step - loss: 1.2676 - accuracy: 0.8248 - val_loss: 0.3705 - val_accuracy: 0.8475
Epoch 774/1000
137/137 [==============================] - 0s 148us/step - loss: 1.2867 - accuracy: 0.8102 - val_loss: 0.3705 - val_accuracy: 0.8475
Epoch 775/1000
137/137 [==============================] - 0s 148us/step - loss: 1.6917 - accuracy: 0.8102 - val_loss: 0.3703 - val_accuracy: 0.8475
Epoch 776/1000
137/137 [==============================] - 0s 146us/step - loss: 1.8052 - accuracy: 0.8029 - val_loss: 0.3702 - val_accuracy: 0.8475
Epoch 777/1000
137/137 [==============================] - 0s 136us/step - loss: 1.4843 - accuracy: 0.8102 - val_loss: 0.3706 - val_accuracy: 0.8475
Epoch 778/1000
137/137 [==============================] - 0s 139us/step - loss: 2.2882 - accuracy: 0.7445 - val_loss: 0.3706 - 

137/137 [==============================] - 0s 128us/step - loss: 1.4951 - accuracy: 0.8029 - val_loss: 0.3652 - val_accuracy: 0.8644
Epoch 828/1000
137/137 [==============================] - 0s 130us/step - loss: 2.0497 - accuracy: 0.7664 - val_loss: 0.3658 - val_accuracy: 0.8644
Epoch 829/1000
137/137 [==============================] - 0s 163us/step - loss: 2.3420 - accuracy: 0.7810 - val_loss: 0.3655 - val_accuracy: 0.8644
Epoch 830/1000
137/137 [==============================] - 0s 136us/step - loss: 1.5781 - accuracy: 0.8175 - val_loss: 0.3660 - val_accuracy: 0.8475
Epoch 831/1000
137/137 [==============================] - 0s 126us/step - loss: 1.7093 - accuracy: 0.7883 - val_loss: 0.3656 - val_accuracy: 0.8644
Epoch 832/1000
137/137 [==============================] - 0s 118us/step - loss: 1.6905 - accuracy: 0.8102 - val_loss: 0.3663 - val_accuracy: 0.8475
Epoch 833/1000
137/137 [==============================] - 0s 135us/step - loss: 2.2701 - accuracy: 0.7518 - val_loss: 0.3668 - 

137/137 [==============================] - 0s 127us/step - loss: 1.3798 - accuracy: 0.8102 - val_loss: 0.3669 - val_accuracy: 0.8644
Epoch 883/1000
137/137 [==============================] - 0s 127us/step - loss: 1.5653 - accuracy: 0.8321 - val_loss: 0.3661 - val_accuracy: 0.8644
Epoch 884/1000
137/137 [==============================] - 0s 134us/step - loss: 2.1431 - accuracy: 0.7664 - val_loss: 0.3670 - val_accuracy: 0.8644
Epoch 885/1000
137/137 [==============================] - 0s 136us/step - loss: 1.9233 - accuracy: 0.7883 - val_loss: 0.3662 - val_accuracy: 0.8644
Epoch 886/1000
137/137 [==============================] - 0s 125us/step - loss: 2.3430 - accuracy: 0.7737 - val_loss: 0.3647 - val_accuracy: 0.8644
Epoch 887/1000
137/137 [==============================] - 0s 134us/step - loss: 1.9905 - accuracy: 0.8102 - val_loss: 0.3648 - val_accuracy: 0.8644
Epoch 888/1000
137/137 [==============================] - 0s 133us/step - loss: 1.0248 - accuracy: 0.8540 - val_loss: 0.3661 - 

137/137 [==============================] - 0s 132us/step - loss: 1.3709 - accuracy: 0.8175 - val_loss: 0.3633 - val_accuracy: 0.8644
Epoch 938/1000
137/137 [==============================] - 0s 130us/step - loss: 1.3842 - accuracy: 0.8102 - val_loss: 0.3635 - val_accuracy: 0.8644
Epoch 939/1000
137/137 [==============================] - 0s 129us/step - loss: 1.6006 - accuracy: 0.7956 - val_loss: 0.3637 - val_accuracy: 0.8644
Epoch 940/1000
137/137 [==============================] - 0s 167us/step - loss: 1.4557 - accuracy: 0.8394 - val_loss: 0.3641 - val_accuracy: 0.8644
Epoch 941/1000
137/137 [==============================] - 0s 153us/step - loss: 1.5466 - accuracy: 0.8394 - val_loss: 0.3630 - val_accuracy: 0.8644
Epoch 942/1000
137/137 [==============================] - 0s 129us/step - loss: 2.1591 - accuracy: 0.7591 - val_loss: 0.3634 - val_accuracy: 0.8644
Epoch 943/1000
137/137 [==============================] - 0s 136us/step - loss: 1.8277 - accuracy: 0.7737 - val_loss: 0.3636 - 

137/137 [==============================] - 0s 138us/step - loss: 1.2628 - accuracy: 0.8248 - val_loss: 0.3691 - val_accuracy: 0.8644
Epoch 993/1000
137/137 [==============================] - 0s 162us/step - loss: 1.9068 - accuracy: 0.7956 - val_loss: 0.3683 - val_accuracy: 0.8644
Epoch 994/1000
137/137 [==============================] - 0s 141us/step - loss: 1.5837 - accuracy: 0.8102 - val_loss: 0.3689 - val_accuracy: 0.8644
Epoch 995/1000
137/137 [==============================] - 0s 199us/step - loss: 1.6860 - accuracy: 0.8029 - val_loss: 0.3688 - val_accuracy: 0.8644
Epoch 996/1000
137/137 [==============================] - 0s 132us/step - loss: 1.4769 - accuracy: 0.8175 - val_loss: 0.3679 - val_accuracy: 0.8644
Epoch 997/1000
137/137 [==============================] - 0s 137us/step - loss: 1.2620 - accuracy: 0.8175 - val_loss: 0.3680 - val_accuracy: 0.8644
Epoch 998/1000
137/137 [==============================] - 0s 128us/step - loss: 2.0226 - accuracy: 0.7883 - val_loss: 0.3672 - 

In [58]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

59/59 [==============================] - 0s 141us/step
combination test accuracy: 86.44%


In [59]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [60]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [61]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 531us/step - loss: 8.7277 - accuracy: 0.5195 - val_loss: 0.6499 - val_accuracy: 0.5657
Epoch 2/1000
231/231 [==============================] - 0s 74us/step - loss: 0.6653 - accuracy: 0.5455 - val_loss: 0.6408 - val_accuracy: 0.5859
Epoch 3/1000
231/231 [==============================] - 0s 69us/step - loss: 0.6367 - accuracy: 0.6190 - val_loss: 0.6282 - val_accuracy: 0.6364
Epoch 4/1000
231/231 [==============================] - 0s 62us/step - loss: 0.6184 - accuracy: 0.6407 - val_loss: 0.6103 - val_accuracy: 0.6162
Epoch 5/1000
231/231 [==============================] - 0s 65us/step - loss: 0.6113 - accuracy: 0.6494 - val_loss: 0.6077 - val_accuracy: 0.6162
Epoch 6/1000
231/231 [==============================] - 0s 57us/step - loss: 0.6074 - accuracy: 0.6407 - val_loss: 0.6039 - val_accuracy: 0.6061
Epoch 7/1000
231/231 [==============================] - 0s 58us/step - loss: 0.5984 

231/231 [==============================] - 0s 154us/step - loss: 0.5044 - accuracy: 0.7403 - val_loss: 0.5257 - val_accuracy: 0.7374
Epoch 57/1000
231/231 [==============================] - 0s 151us/step - loss: 0.4998 - accuracy: 0.7229 - val_loss: 0.5180 - val_accuracy: 0.7374
Epoch 58/1000
231/231 [==============================] - 0s 140us/step - loss: 0.4947 - accuracy: 0.7229 - val_loss: 0.5149 - val_accuracy: 0.7475
Epoch 59/1000
231/231 [==============================] - 0s 113us/step - loss: 0.4893 - accuracy: 0.7446 - val_loss: 0.5168 - val_accuracy: 0.7374
Epoch 60/1000
231/231 [==============================] - 0s 95us/step - loss: 0.4869 - accuracy: 0.7403 - val_loss: 0.5198 - val_accuracy: 0.7374
Epoch 61/1000
231/231 [==============================] - 0s 92us/step - loss: 0.4869 - accuracy: 0.7446 - val_loss: 0.5113 - val_accuracy: 0.7273
Epoch 62/1000
231/231 [==============================] - 0s 106us/step - loss: 0.4884 - accuracy: 0.7316 - val_loss: 0.5216 - val_accu

Epoch 112/1000
231/231 [==============================] - 0s 105us/step - loss: 0.4539 - accuracy: 0.7706 - val_loss: 0.4996 - val_accuracy: 0.7576
Epoch 113/1000
231/231 [==============================] - 0s 97us/step - loss: 0.4542 - accuracy: 0.7662 - val_loss: 0.5126 - val_accuracy: 0.7778
Epoch 114/1000
231/231 [==============================] - 0s 104us/step - loss: 0.4556 - accuracy: 0.7835 - val_loss: 0.5142 - val_accuracy: 0.7576
Epoch 115/1000
231/231 [==============================] - 0s 116us/step - loss: 0.4505 - accuracy: 0.7749 - val_loss: 0.4949 - val_accuracy: 0.7879
Epoch 116/1000
231/231 [==============================] - 0s 95us/step - loss: 0.4423 - accuracy: 0.7879 - val_loss: 0.5262 - val_accuracy: 0.7677
Epoch 117/1000
231/231 [==============================] - 0s 105us/step - loss: 0.4563 - accuracy: 0.7706 - val_loss: 0.4997 - val_accuracy: 0.7778
Epoch 118/1000
231/231 [==============================] - 0s 95us/step - loss: 0.4567 - accuracy: 0.7706 - val_los

231/231 [==============================] - 0s 87us/step - loss: 0.4214 - accuracy: 0.7835 - val_loss: 0.5038 - val_accuracy: 0.7980
Epoch 168/1000
231/231 [==============================] - 0s 91us/step - loss: 0.4226 - accuracy: 0.7922 - val_loss: 0.5144 - val_accuracy: 0.7677
Epoch 169/1000
231/231 [==============================] - 0s 110us/step - loss: 0.4347 - accuracy: 0.7662 - val_loss: 0.5069 - val_accuracy: 0.7677
Epoch 170/1000
231/231 [==============================] - 0s 93us/step - loss: 0.4293 - accuracy: 0.7792 - val_loss: 0.4881 - val_accuracy: 0.7879
Epoch 171/1000
231/231 [==============================] - 0s 120us/step - loss: 0.4217 - accuracy: 0.7749 - val_loss: 0.4967 - val_accuracy: 0.7879
Epoch 172/1000
231/231 [==============================] - 0s 94us/step - loss: 0.4256 - accuracy: 0.7922 - val_loss: 0.5018 - val_accuracy: 0.7374
Epoch 173/1000
231/231 [==============================] - 0s 99us/step - loss: 0.4252 - accuracy: 0.7879 - val_loss: 0.5036 - val_a

Epoch 223/1000
231/231 [==============================] - 0s 99us/step - loss: 0.4046 - accuracy: 0.7922 - val_loss: 0.4951 - val_accuracy: 0.7879
Epoch 224/1000
231/231 [==============================] - 0s 128us/step - loss: 0.4001 - accuracy: 0.7879 - val_loss: 0.4954 - val_accuracy: 0.7778
Epoch 225/1000
231/231 [==============================] - 0s 106us/step - loss: 0.3994 - accuracy: 0.7879 - val_loss: 0.4966 - val_accuracy: 0.7677
Epoch 226/1000
231/231 [==============================] - 0s 90us/step - loss: 0.3998 - accuracy: 0.7922 - val_loss: 0.4990 - val_accuracy: 0.7475
Epoch 227/1000
231/231 [==============================] - 0s 101us/step - loss: 0.4028 - accuracy: 0.8009 - val_loss: 0.4925 - val_accuracy: 0.7677
Epoch 228/1000
231/231 [==============================] - 0s 92us/step - loss: 0.4000 - accuracy: 0.7879 - val_loss: 0.4903 - val_accuracy: 0.7778
Epoch 229/1000
231/231 [==============================] - 0s 126us/step - loss: 0.3992 - accuracy: 0.7835 - val_los

...

Epoch 775/1000
231/231 [==============================] - 0s 105us/step - loss: 0.3653 - accuracy: 0.8095 - val_loss: 0.5662 - val_accuracy: 0.7475
Epoch 776/1000
231/231 [==============================] - 0s 130us/step - loss: 0.3128 - accuracy: 0.8398 - val_loss: 0.5449 - val_accuracy: 0.7475
Epoch 777/1000
231/231 [==============================] - 0s 94us/step - loss: 0.3020 - accuracy: 0.8312 - val_loss: 0.5496 - val_accuracy: 0.7576
Epoch 778/1000
231/231 [==============================] - 0s 103us/step - loss: 0.3143 - accuracy: 0.8312 - val_loss: 0.5586 - val_accuracy: 0.7475
Epoch 779/1000
231/231 [==============================] - 0s 107us/step - loss: 0.3115 - accuracy: 0.8442 - val_loss: 0.5411 - val_accuracy: 0.7879
Epoch 780/1000
231/231 [==============================] - 0s 102us/step - loss: 0.3081 - accuracy: 0.8398 - val_loss: 0.5577 - val_accuracy: 0.7475
Epoch 781/1000
231/231 [==============================] - 0s 95us/step - loss: 0.3050 - accuracy: 0.8442 - val_lo

231/231 [==============================] - 0s 93us/step - loss: 0.3483 - accuracy: 0.8139 - val_loss: 0.5989 - val_accuracy: 0.7475
Epoch 831/1000
231/231 [==============================] - 0s 103us/step - loss: 0.3511 - accuracy: 0.8095 - val_loss: 0.5990 - val_accuracy: 0.7576
Epoch 832/1000
231/231 [==============================] - 0s 101us/step - loss: 0.3450 - accuracy: 0.8398 - val_loss: 0.6031 - val_accuracy: 0.7273
Epoch 833/1000
231/231 [==============================] - 0s 96us/step - loss: 0.3445 - accuracy: 0.8442 - val_loss: 0.6104 - val_accuracy: 0.7475
Epoch 834/1000
231/231 [==============================] - 0s 106us/step - loss: 0.3420 - accuracy: 0.8355 - val_loss: 0.6032 - val_accuracy: 0.7374
Epoch 835/1000
231/231 [==============================] - 0s 134us/step - loss: 0.3458 - accuracy: 0.8095 - val_loss: 0.5804 - val_accuracy: 0.7576
Epoch 836/1000
231/231 [==============================] - 0s 90us/step - loss: 0.3223 - accuracy: 0.8182 - val_loss: 0.5705 - val

Epoch 886/1000
231/231 [==============================] - 0s 94us/step - loss: 0.2951 - accuracy: 0.8485 - val_loss: 0.5514 - val_accuracy: 0.7677
Epoch 887/1000
231/231 [==============================] - 0s 99us/step - loss: 0.3001 - accuracy: 0.8398 - val_loss: 0.5664 - val_accuracy: 0.7576
Epoch 888/1000
231/231 [==============================] - 0s 100us/step - loss: 0.3002 - accuracy: 0.8442 - val_loss: 0.5633 - val_accuracy: 0.7677
Epoch 889/1000
231/231 [==============================] - 0s 96us/step - loss: 0.2899 - accuracy: 0.8442 - val_loss: 0.5660 - val_accuracy: 0.7475
Epoch 890/1000
231/231 [==============================] - 0s 92us/step - loss: 0.3143 - accuracy: 0.8398 - val_loss: 0.5537 - val_accuracy: 0.7677
Epoch 891/1000
231/231 [==============================] - 0s 160us/step - loss: 0.2893 - accuracy: 0.8398 - val_loss: 0.5525 - val_accuracy: 0.7778
Epoch 892/1000
231/231 [==============================] - 0s 125us/step - loss: 0.3223 - accuracy: 0.8268 - val_loss

231/231 [==============================] - 0s 95us/step - loss: 0.3016 - accuracy: 0.8528 - val_loss: 0.5756 - val_accuracy: 0.7576
Epoch 942/1000
231/231 [==============================] - 0s 96us/step - loss: 0.2999 - accuracy: 0.8442 - val_loss: 0.5647 - val_accuracy: 0.7576
Epoch 943/1000
231/231 [==============================] - 0s 105us/step - loss: 0.2873 - accuracy: 0.8485 - val_loss: 0.5555 - val_accuracy: 0.7677
Epoch 944/1000
231/231 [==============================] - 0s 97us/step - loss: 0.2959 - accuracy: 0.8398 - val_loss: 0.5668 - val_accuracy: 0.7677
Epoch 945/1000
231/231 [==============================] - 0s 114us/step - loss: 0.2891 - accuracy: 0.8485 - val_loss: 0.5976 - val_accuracy: 0.7475
Epoch 946/1000
231/231 [==============================] - 0s 103us/step - loss: 0.3572 - accuracy: 0.8225 - val_loss: 0.5858 - val_accuracy: 0.7475
Epoch 947/1000
231/231 [==============================] - 0s 85us/step - loss: 0.2957 - accuracy: 0.8442 - val_loss: 0.5591 - val_

Epoch 997/1000
231/231 [==============================] - 0s 98us/step - loss: 0.2768 - accuracy: 0.8528 - val_loss: 0.5707 - val_accuracy: 0.7576
Epoch 998/1000
231/231 [==============================] - 0s 102us/step - loss: 0.2713 - accuracy: 0.8528 - val_loss: 0.5684 - val_accuracy: 0.7576
Epoch 999/1000
231/231 [==============================] - 0s 102us/step - loss: 0.2745 - accuracy: 0.8528 - val_loss: 0.5699 - val_accuracy: 0.7576
Epoch 1000/1000
231/231 [==============================] - 0s 110us/step - loss: 0.2732 - accuracy: 0.8571 - val_loss: 0.5705 - val_accuracy: 0.7576


In [62]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

99/99 [==============================] - 0s 90us/step
over-sampling test accuracy: 75.76%


In [67]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [68]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [69]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 497us/step - loss: 12.8537 - accuracy: 0.5195 - val_loss: 0.7592 - val_accuracy: 0.4646
Epoch 2/1000
231/231 [==============================] - 0s 57us/step - loss: 2.1529 - accuracy: 0.4675 - val_loss: 6.7066 - val_accuracy: 0.4949
Epoch 3/1000
231/231 [==============================] - 0s 80us/step - loss: 7.6896 - accuracy: 0.5022 - val_loss: 0.7059 - val_accuracy: 0.3636
Epoch 4/1000
231/231 [==============================] - 0s 94us/step - loss: 0.6958 - accuracy: 0.4502 - val_loss: 0.7016 - val_accuracy: 0.3838
Epoch 5/1000
231/231 [==============================] - 0s 80us/step - loss: 0.6915 - accuracy: 0.4632 - val_loss: 0.6978 - val_accuracy: 0.4141
Epoch 6/1000
231/231 [==============================] - 0s 107us/step - loss: 0.6882 - accuracy: 0.4805 - val_loss: 0.6945 - val_accuracy: 0.4545
Epoch 7/1000
231/231 [==============================] - 0s 80us/step - loss: 0.685

Epoch 57/1000
231/231 [==============================] - 0s 85us/step - loss: 0.5793 - accuracy: 0.6623 - val_loss: 0.5835 - val_accuracy: 0.6768
Epoch 58/1000
231/231 [==============================] - 0s 64us/step - loss: 0.5738 - accuracy: 0.6710 - val_loss: 0.5821 - val_accuracy: 0.6768
Epoch 59/1000
231/231 [==============================] - 0s 63us/step - loss: 0.5663 - accuracy: 0.6623 - val_loss: 0.5811 - val_accuracy: 0.6768
Epoch 60/1000
231/231 [==============================] - 0s 84us/step - loss: 0.5650 - accuracy: 0.6710 - val_loss: 0.5864 - val_accuracy: 0.7475
Epoch 61/1000
231/231 [==============================] - 0s 78us/step - loss: 0.5669 - accuracy: 0.6623 - val_loss: 0.5782 - val_accuracy: 0.7576
Epoch 62/1000
231/231 [==============================] - 0s 68us/step - loss: 0.5620 - accuracy: 0.6753 - val_loss: 0.5774 - val_accuracy: 0.7576
Epoch 63/1000
231/231 [==============================] - 0s 63us/step - loss: 0.5613 - accuracy: 0.6797 - val_loss: 0.5768 -

Epoch 113/1000
231/231 [==============================] - 0s 66us/step - loss: 0.5027 - accuracy: 0.7359 - val_loss: 0.5405 - val_accuracy: 0.7677
Epoch 114/1000
231/231 [==============================] - 0s 70us/step - loss: 0.5350 - accuracy: 0.7316 - val_loss: 0.5770 - val_accuracy: 0.7273
Epoch 115/1000
231/231 [==============================] - 0s 72us/step - loss: 0.5185 - accuracy: 0.7403 - val_loss: 0.5334 - val_accuracy: 0.7677
Epoch 116/1000
231/231 [==============================] - 0s 65us/step - loss: 0.5035 - accuracy: 0.7229 - val_loss: 0.5327 - val_accuracy: 0.7576
Epoch 117/1000
231/231 [==============================] - 0s 69us/step - loss: 0.4999 - accuracy: 0.7446 - val_loss: 0.5315 - val_accuracy: 0.7677
Epoch 118/1000
231/231 [==============================] - 0s 63us/step - loss: 0.4984 - accuracy: 0.7359 - val_loss: 0.5343 - val_accuracy: 0.7677
Epoch 119/1000
231/231 [==============================] - 0s 66us/step - loss: 0.5141 - accuracy: 0.7229 - val_loss: 0

Epoch 169/1000
231/231 [==============================] - 0s 71us/step - loss: 0.4760 - accuracy: 0.7446 - val_loss: 0.5850 - val_accuracy: 0.7071
Epoch 170/1000
231/231 [==============================] - 0s 77us/step - loss: 0.5443 - accuracy: 0.7273 - val_loss: 0.5271 - val_accuracy: 0.7273
Epoch 171/1000
231/231 [==============================] - 0s 71us/step - loss: 0.4656 - accuracy: 0.7446 - val_loss: 0.5260 - val_accuracy: 0.7677
Epoch 172/1000
231/231 [==============================] - 0s 68us/step - loss: 0.4697 - accuracy: 0.7576 - val_loss: 0.5142 - val_accuracy: 0.7677
Epoch 173/1000
231/231 [==============================] - 0s 66us/step - loss: 0.5085 - accuracy: 0.7446 - val_loss: 0.5158 - val_accuracy: 0.7778
Epoch 174/1000
231/231 [==============================] - 0s 75us/step - loss: 0.4826 - accuracy: 0.7446 - val_loss: 0.5168 - val_accuracy: 0.7778
Epoch 175/1000
231/231 [==============================] - 0s 69us/step - loss: 0.4659 - accuracy: 0.7619 - val_loss: 0

231/231 [==============================] - 0s 68us/step - loss: 0.4490 - accuracy: 0.7749 - val_loss: 0.5143 - val_accuracy: 0.7374
Epoch 225/1000
231/231 [==============================] - 0s 70us/step - loss: 0.4421 - accuracy: 0.7662 - val_loss: 0.5076 - val_accuracy: 0.7475
Epoch 226/1000
231/231 [==============================] - 0s 73us/step - loss: 0.4411 - accuracy: 0.7749 - val_loss: 0.5297 - val_accuracy: 0.7576
Epoch 227/1000
231/231 [==============================] - 0s 82us/step - loss: 0.4749 - accuracy: 0.7706 - val_loss: 0.5156 - val_accuracy: 0.7374
Epoch 228/1000
231/231 [==============================] - 0s 83us/step - loss: 0.4354 - accuracy: 0.7749 - val_loss: 0.5109 - val_accuracy: 0.7374
Epoch 229/1000
231/231 [==============================] - 0s 57us/step - loss: 0.4353 - accuracy: 0.7706 - val_loss: 0.5060 - val_accuracy: 0.7879
Epoch 230/1000
231/231 [==============================] - 0s 61us/step - loss: 0.4426 - accuracy: 0.7749 - val_loss: 0.5129 - val_acc

...

Epoch 726/1000
231/231 [==============================] - 0s 68us/step - loss: 0.2991 - accuracy: 0.8571 - val_loss: 0.5273 - val_accuracy: 0.7879
Epoch 727/1000
231/231 [==============================] - 0s 73us/step - loss: 0.3207 - accuracy: 0.8442 - val_loss: 0.5746 - val_accuracy: 0.7071
Epoch 728/1000
231/231 [==============================] - 0s 72us/step - loss: 0.3242 - accuracy: 0.8442 - val_loss: 0.5220 - val_accuracy: 0.7778
Epoch 729/1000
231/231 [==============================] - 0s 59us/step - loss: 0.2958 - accuracy: 0.8571 - val_loss: 0.5351 - val_accuracy: 0.7677
Epoch 730/1000
231/231 [==============================] - 0s 77us/step - loss: 0.3238 - accuracy: 0.8225 - val_loss: 0.5266 - val_accuracy: 0.7778
Epoch 731/1000
231/231 [==============================] - 0s 83us/step - loss: 0.3066 - accuracy: 0.8485 - val_loss: 0.6079 - val_accuracy: 0.7475
Epoch 732/1000
231/231 [==============================] - 0s 78us/step - loss: 0.3165 - accuracy: 0.8442 - val_loss: 0

Epoch 782/1000
231/231 [==============================] - 0s 63us/step - loss: 0.2882 - accuracy: 0.8571 - val_loss: 0.5288 - val_accuracy: 0.7778
Epoch 783/1000
231/231 [==============================] - 0s 72us/step - loss: 0.3653 - accuracy: 0.8312 - val_loss: 0.7676 - val_accuracy: 0.7071
Epoch 784/1000
231/231 [==============================] - 0s 79us/step - loss: 0.3656 - accuracy: 0.8225 - val_loss: 0.5401 - val_accuracy: 0.7778
Epoch 785/1000
231/231 [==============================] - 0s 66us/step - loss: 0.2934 - accuracy: 0.8571 - val_loss: 0.5285 - val_accuracy: 0.7980
Epoch 786/1000
231/231 [==============================] - 0s 73us/step - loss: 0.2893 - accuracy: 0.8615 - val_loss: 0.5532 - val_accuracy: 0.7576
Epoch 787/1000
231/231 [==============================] - 0s 68us/step - loss: 0.2894 - accuracy: 0.8615 - val_loss: 0.5304 - val_accuracy: 0.7778
Epoch 788/1000
231/231 [==============================] - 0s 72us/step - loss: 0.2879 - accuracy: 0.8615 - val_loss: 0

Epoch 838/1000
231/231 [==============================] - 0s 69us/step - loss: 0.3136 - accuracy: 0.8528 - val_loss: 0.5533 - val_accuracy: 0.7980
Epoch 839/1000
231/231 [==============================] - 0s 73us/step - loss: 0.3270 - accuracy: 0.8312 - val_loss: 0.6036 - val_accuracy: 0.7374
Epoch 840/1000
231/231 [==============================] - 0s 74us/step - loss: 0.3664 - accuracy: 0.8095 - val_loss: 0.6025 - val_accuracy: 0.7374
Epoch 841/1000
231/231 [==============================] - 0s 65us/step - loss: 0.3510 - accuracy: 0.8268 - val_loss: 0.5530 - val_accuracy: 0.7980
Epoch 842/1000
231/231 [==============================] - 0s 68us/step - loss: 0.3109 - accuracy: 0.8615 - val_loss: 0.5510 - val_accuracy: 0.7677
Epoch 843/1000
231/231 [==============================] - 0s 81us/step - loss: 0.3084 - accuracy: 0.8571 - val_loss: 0.5577 - val_accuracy: 0.7374
Epoch 844/1000
231/231 [==============================] - 0s 65us/step - loss: 0.3125 - accuracy: 0.8398 - val_loss: 0

231/231 [==============================] - 0s 70us/step - loss: 0.2780 - accuracy: 0.8571 - val_loss: 0.5548 - val_accuracy: 0.7677
Epoch 894/1000
231/231 [==============================] - 0s 72us/step - loss: 0.2766 - accuracy: 0.8615 - val_loss: 0.5423 - val_accuracy: 0.7576
Epoch 895/1000
231/231 [==============================] - 0s 84us/step - loss: 0.2766 - accuracy: 0.8615 - val_loss: 0.5503 - val_accuracy: 0.7778
Epoch 896/1000
231/231 [==============================] - 0s 77us/step - loss: 0.2756 - accuracy: 0.8571 - val_loss: 0.5591 - val_accuracy: 0.7374
Epoch 897/1000
231/231 [==============================] - 0s 74us/step - loss: 0.3288 - accuracy: 0.8312 - val_loss: 0.5489 - val_accuracy: 0.7778
Epoch 898/1000
231/231 [==============================] - 0s 66us/step - loss: 0.2762 - accuracy: 0.8615 - val_loss: 0.5456 - val_accuracy: 0.7879
Epoch 899/1000
231/231 [==============================] - 0s 74us/step - loss: 0.2852 - accuracy: 0.8485 - val_loss: 0.6255 - val_acc

Epoch 949/1000
231/231 [==============================] - 0s 75us/step - loss: 0.2717 - accuracy: 0.8615 - val_loss: 0.5516 - val_accuracy: 0.7778
Epoch 950/1000
231/231 [==============================] - 0s 82us/step - loss: 0.2702 - accuracy: 0.8615 - val_loss: 0.5548 - val_accuracy: 0.7677
Epoch 951/1000
231/231 [==============================] - 0s 73us/step - loss: 0.2685 - accuracy: 0.8615 - val_loss: 0.5496 - val_accuracy: 0.7778
Epoch 952/1000
231/231 [==============================] - 0s 68us/step - loss: 0.2692 - accuracy: 0.8615 - val_loss: 0.5673 - val_accuracy: 0.7778
Epoch 953/1000
231/231 [==============================] - 0s 73us/step - loss: 0.2810 - accuracy: 0.8571 - val_loss: 0.5529 - val_accuracy: 0.7677
Epoch 954/1000
231/231 [==============================] - 0s 99us/step - loss: 0.3246 - accuracy: 0.8268 - val_loss: 0.5533 - val_accuracy: 0.7778
Epoch 955/1000
231/231 [==============================] - 0s 74us/step - loss: 0.2714 - accuracy: 0.8615 - val_loss: 0

In [70]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

99/99 [==============================] - 0s 105us/step
over-sampling test accuracy: 75.76%


In [71]:
############ Feature selection using lasso ##########

In [74]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [75]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [76]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [77]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [78]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 373
selected features: 26


In [79]:
cols = sel_features.values
cols.reshape((1, -1))

array([[  0,  30,  64,  77,  81,  83,  93,  95, 117, 118, 124, 152, 178,
        189, 203, 209, 212, 213, 222, 224, 226, 250, 296, 335, 371, 372]])

In [80]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTTAT',
       'TTATGTTTCTGGA', 'TCGTGTCTGT', 'TCACACCGCCT', 'TATCAGTTTTGTT',
       'TAGATTCAAATAT',
       'TAACAAAACTGATAATTAATATACTGCTTATCAATAACATTAAGCGGTGGTGAAACTTCATCATT',
       'TAAATCGTCTTTATATTTAATTATTAAATTAACAAATTTTAATTGGCGGATGAGGTATCCAGTTACCTCGTTCGCCAATTATTTTTCGCAATATAAAAAG',
       'GTGAATTCATG', 'GTCTGATTTTTT', 'GTAGTAGATAATTTT', 'CTAATCCTTCAAT',
       'ACTTTTTATATTGCGAAAAATAATTGGCGAACGAGGTAACTGGATACCTCATCCGCCAATTAAAATTTGTTAATTTAATAATTAAATATAAAGACGATTT',
       'X1_725392_T_A', 'fnbA', 'group_6584', 'scc', 'group_662',
       'yezG_1', 'group_3025', 'group_4981', 'group_4761', 'group_7718',
       'group_2541', 'ST', 'CC'], dtype='<U100')

In [81]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            361, 362, 363, 364, 365, 366, 367, 368, 369, 370],
           dtype='int64', length=347)

In [82]:
len(removed_features)

347

In [83]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 26) (255,) (255, 27)


In [84]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 106), (1, 89)]


In [85]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 165), (1, 165)]


In [86]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel)

In [87]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [89]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [95]:
#### neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [96]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [97]:
model2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 569us/step - loss: 20.9864 - accuracy: 0.5337 - val_loss: 0.6519 - val_accuracy: 0.5065
Epoch 2/1000
178/178 [==============================] - 0s 78us/step - loss: 0.6629 - accuracy: 0.5899 - val_loss: 0.7327 - val_accuracy: 0.6494
Epoch 3/1000
178/178 [==============================] - 0s 104us/step - loss: 0.7120 - accuracy: 0.6573 - val_loss: 0.6318 - val_accuracy: 0.6623
Epoch 4/1000
178/178 [==============================] - 0s 97us/step - loss: 0.6274 - accuracy: 0.7247 - val_loss: 0.6277 - val_accuracy: 0.6623
Epoch 5/1000
178/178 [==============================] - 0s 117us/step - loss: 0.6246 - accuracy: 0.7079 - val_loss: 0.6301 - val_accuracy: 0.6623
Epoch 6/1000
178/178 [==============================] - 0s 134us/step - loss: 0.6440 - accuracy: 0.7079 - val_loss: 0.6377 - val_accuracy: 0.6623
Epoch 7/1000
178/178 [==============================] - 0s 104us/step - loss: 0.

178/178 [==============================] - 0s 112us/step - loss: 0.4919 - accuracy: 0.8034 - val_loss: 0.5859 - val_accuracy: 0.7273
Epoch 57/1000
178/178 [==============================] - 0s 105us/step - loss: 0.5042 - accuracy: 0.7978 - val_loss: 0.6080 - val_accuracy: 0.7403
Epoch 58/1000
178/178 [==============================] - 0s 107us/step - loss: 0.4760 - accuracy: 0.8090 - val_loss: 0.5943 - val_accuracy: 0.7403
Epoch 59/1000
178/178 [==============================] - 0s 89us/step - loss: 0.5159 - accuracy: 0.8034 - val_loss: 1.1166 - val_accuracy: 0.5455
Epoch 60/1000
178/178 [==============================] - 0s 119us/step - loss: 0.5351 - accuracy: 0.7865 - val_loss: 0.6666 - val_accuracy: 0.7532
Epoch 61/1000
178/178 [==============================] - 0s 98us/step - loss: 0.5660 - accuracy: 0.7921 - val_loss: 0.5886 - val_accuracy: 0.7403
Epoch 62/1000
178/178 [==============================] - 0s 86us/step - loss: 0.5081 - accuracy: 0.7978 - val_loss: 0.6172 - val_accur

178/178 [==============================] - 0s 110us/step - loss: 0.4439 - accuracy: 0.8090 - val_loss: 0.6382 - val_accuracy: 0.7532
Epoch 112/1000
178/178 [==============================] - 0s 108us/step - loss: 0.4752 - accuracy: 0.8258 - val_loss: 0.5846 - val_accuracy: 0.7532
Epoch 113/1000
178/178 [==============================] - 0s 105us/step - loss: 0.4916 - accuracy: 0.8090 - val_loss: 0.5873 - val_accuracy: 0.7662
Epoch 114/1000
178/178 [==============================] - 0s 105us/step - loss: 0.5417 - accuracy: 0.7978 - val_loss: 0.5997 - val_accuracy: 0.7532
Epoch 115/1000
178/178 [==============================] - 0s 111us/step - loss: 0.4751 - accuracy: 0.8090 - val_loss: 0.6568 - val_accuracy: 0.7662
Epoch 116/1000
178/178 [==============================] - 0s 162us/step - loss: 0.4612 - accuracy: 0.8034 - val_loss: 0.6485 - val_accuracy: 0.7532
Epoch 117/1000
178/178 [==============================] - 0s 131us/step - loss: 0.4962 - accuracy: 0.8090 - val_loss: 0.6183 - 

178/178 [==============================] - 0s 90us/step - loss: 0.4371 - accuracy: 0.8090 - val_loss: 0.5983 - val_accuracy: 0.7532
Epoch 167/1000
178/178 [==============================] - 0s 132us/step - loss: 0.4525 - accuracy: 0.8090 - val_loss: 0.5938 - val_accuracy: 0.7662
Epoch 168/1000
178/178 [==============================] - 0s 113us/step - loss: 0.4431 - accuracy: 0.8146 - val_loss: 0.5985 - val_accuracy: 0.7532
Epoch 169/1000
178/178 [==============================] - 0s 105us/step - loss: 0.4763 - accuracy: 0.7978 - val_loss: 0.5882 - val_accuracy: 0.7662
Epoch 170/1000
178/178 [==============================] - 0s 89us/step - loss: 0.4452 - accuracy: 0.8090 - val_loss: 0.5893 - val_accuracy: 0.7792
Epoch 171/1000
178/178 [==============================] - 0s 97us/step - loss: 0.4251 - accuracy: 0.8202 - val_loss: 0.6311 - val_accuracy: 0.7662
Epoch 172/1000
178/178 [==============================] - 0s 84us/step - loss: 0.4983 - accuracy: 0.8090 - val_loss: 0.5872 - val_

Epoch 222/1000
178/178 [==============================] - 0s 112us/step - loss: 0.4475 - accuracy: 0.8202 - val_loss: 0.5722 - val_accuracy: 0.7662
Epoch 223/1000
178/178 [==============================] - 0s 100us/step - loss: 0.4456 - accuracy: 0.8315 - val_loss: 0.5748 - val_accuracy: 0.7532
Epoch 224/1000
178/178 [==============================] - 0s 105us/step - loss: 0.4439 - accuracy: 0.8146 - val_loss: 0.5771 - val_accuracy: 0.7532
Epoch 225/1000
178/178 [==============================] - 0s 92us/step - loss: 0.4372 - accuracy: 0.8202 - val_loss: 0.5788 - val_accuracy: 0.7532
Epoch 226/1000
178/178 [==============================] - 0s 87us/step - loss: 0.4422 - accuracy: 0.8202 - val_loss: 0.5855 - val_accuracy: 0.7662
Epoch 227/1000
178/178 [==============================] - 0s 104us/step - loss: 0.4425 - accuracy: 0.8258 - val_loss: 0.5843 - val_accuracy: 0.7532
Epoch 228/1000
178/178 [==============================] - 0s 100us/step - loss: 0.4376 - accuracy: 0.8090 - val_lo

...

178/178 [==============================] - 0s 112us/step - loss: 0.4000 - accuracy: 0.8315 - val_loss: 0.6078 - val_accuracy: 0.7532
Epoch 775/1000
178/178 [==============================] - 0s 110us/step - loss: 0.4062 - accuracy: 0.8146 - val_loss: 0.5982 - val_accuracy: 0.7662
Epoch 776/1000
178/178 [==============================] - 0s 95us/step - loss: 0.3959 - accuracy: 0.8258 - val_loss: 0.6158 - val_accuracy: 0.7662
Epoch 777/1000
178/178 [==============================] - 0s 102us/step - loss: 0.3987 - accuracy: 0.8315 - val_loss: 0.6050 - val_accuracy: 0.7662
Epoch 778/1000
178/178 [==============================] - 0s 97us/step - loss: 0.3887 - accuracy: 0.8315 - val_loss: 0.6769 - val_accuracy: 0.7662
Epoch 779/1000
178/178 [==============================] - 0s 89us/step - loss: 0.4505 - accuracy: 0.8090 - val_loss: 0.6086 - val_accuracy: 0.7662
Epoch 780/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3886 - accuracy: 0.8258 - val_loss: 0.6191 - val_

Epoch 830/1000
178/178 [==============================] - 0s 75us/step - loss: 0.4109 - accuracy: 0.8258 - val_loss: 0.6022 - val_accuracy: 0.7532
Epoch 831/1000
178/178 [==============================] - 0s 72us/step - loss: 0.3935 - accuracy: 0.8202 - val_loss: 0.5994 - val_accuracy: 0.7662
Epoch 832/1000
178/178 [==============================] - 0s 106us/step - loss: 0.3888 - accuracy: 0.8258 - val_loss: 0.6033 - val_accuracy: 0.7662
Epoch 833/1000
178/178 [==============================] - 0s 118us/step - loss: 0.3870 - accuracy: 0.8315 - val_loss: 0.6094 - val_accuracy: 0.7662
Epoch 834/1000
178/178 [==============================] - 0s 101us/step - loss: 0.3865 - accuracy: 0.8258 - val_loss: 0.6008 - val_accuracy: 0.7662
Epoch 835/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3895 - accuracy: 0.8202 - val_loss: 0.6124 - val_accuracy: 0.7662
Epoch 836/1000
178/178 [==============================] - 0s 121us/step - loss: 0.4242 - accuracy: 0.8258 - val_los

178/178 [==============================] - 0s 96us/step - loss: 0.3990 - accuracy: 0.8202 - val_loss: 0.6043 - val_accuracy: 0.7662
Epoch 886/1000
178/178 [==============================] - 0s 95us/step - loss: 0.3867 - accuracy: 0.8371 - val_loss: 0.6035 - val_accuracy: 0.7662
Epoch 887/1000
178/178 [==============================] - 0s 91us/step - loss: 0.3925 - accuracy: 0.8258 - val_loss: 0.6055 - val_accuracy: 0.7662
Epoch 888/1000
178/178 [==============================] - 0s 81us/step - loss: 0.4093 - accuracy: 0.8202 - val_loss: 0.6003 - val_accuracy: 0.7662
Epoch 889/1000
178/178 [==============================] - 0s 94us/step - loss: 0.4029 - accuracy: 0.8202 - val_loss: 0.6022 - val_accuracy: 0.7662
Epoch 890/1000
178/178 [==============================] - 0s 107us/step - loss: 0.4120 - accuracy: 0.8258 - val_loss: 0.6120 - val_accuracy: 0.7662
Epoch 891/1000
178/178 [==============================] - 0s 85us/step - loss: 0.3968 - accuracy: 0.8258 - val_loss: 0.6065 - val_ac

178/178 [==============================] - 0s 100us/step - loss: 0.3805 - accuracy: 0.8371 - val_loss: 0.6036 - val_accuracy: 0.7662
Epoch 941/1000
178/178 [==============================] - 0s 97us/step - loss: 0.3872 - accuracy: 0.8258 - val_loss: 0.6002 - val_accuracy: 0.7662
Epoch 942/1000
178/178 [==============================] - 0s 104us/step - loss: 0.3834 - accuracy: 0.8315 - val_loss: 0.6003 - val_accuracy: 0.7662
Epoch 943/1000
178/178 [==============================] - 0s 115us/step - loss: 0.3875 - accuracy: 0.8258 - val_loss: 0.6055 - val_accuracy: 0.7662
Epoch 944/1000
178/178 [==============================] - 0s 115us/step - loss: 0.3862 - accuracy: 0.8258 - val_loss: 0.6016 - val_accuracy: 0.7662
Epoch 945/1000
178/178 [==============================] - 0s 111us/step - loss: 0.3814 - accuracy: 0.8371 - val_loss: 0.6098 - val_accuracy: 0.7662
Epoch 946/1000
178/178 [==============================] - 0s 105us/step - loss: 0.3857 - accuracy: 0.8371 - val_loss: 0.6060 - v

Epoch 996/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3846 - accuracy: 0.8315 - val_loss: 0.6297 - val_accuracy: 0.7532
Epoch 997/1000
178/178 [==============================] - 0s 105us/step - loss: 0.3871 - accuracy: 0.8315 - val_loss: 0.6225 - val_accuracy: 0.7532
Epoch 998/1000
178/178 [==============================] - 0s 84us/step - loss: 0.3854 - accuracy: 0.8315 - val_loss: 0.6079 - val_accuracy: 0.7662
Epoch 999/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3812 - accuracy: 0.8315 - val_loss: 0.6073 - val_accuracy: 0.7792
Epoch 1000/1000
178/178 [==============================] - 0s 101us/step - loss: 0.3887 - accuracy: 0.8258 - val_loss: 0.6334 - val_accuracy: 0.7532


In [98]:
acc_test = model2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test*100))



77/77 [==============================] - 0s 105us/step
test accuracy: 75.32%


In [119]:
#### improve neural network
model2_2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [120]:
model2_2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [121]:
model2_2.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 634us/step - loss: 9.1259 - accuracy: 0.5337 - val_loss: 3.0398 - val_accuracy: 0.7013
Epoch 2/1000
178/178 [==============================] - 0s 81us/step - loss: 3.7718 - accuracy: 0.5449 - val_loss: 2.3463 - val_accuracy: 0.4156
Epoch 3/1000
178/178 [==============================] - 0s 132us/step - loss: 1.4523 - accuracy: 0.4213 - val_loss: 0.6949 - val_accuracy: 0.5844
Epoch 4/1000
178/178 [==============================] - 0s 109us/step - loss: 0.7213 - accuracy: 0.4157 - val_loss: 0.6588 - val_accuracy: 0.7273
Epoch 5/1000
178/178 [==============================] - 0s 118us/step - loss: 0.6859 - accuracy: 0.6067 - val_loss: 0.6505 - val_accuracy: 0.7143
Epoch 6/1000
178/178 [==============================] - 0s 117us/step - loss: 0.6835 - accuracy: 0.6124 - val_loss: 0.6498 - val_accuracy: 0.7013
Epoch 7/1000
178/178 [==============================] - 0s 129us/step - loss: 0.

178/178 [==============================] - 0s 103us/step - loss: 0.5499 - accuracy: 0.7865 - val_loss: 0.5806 - val_accuracy: 0.7143
Epoch 57/1000
178/178 [==============================] - 0s 104us/step - loss: 0.5518 - accuracy: 0.7865 - val_loss: 0.5807 - val_accuracy: 0.7143
Epoch 58/1000
178/178 [==============================] - 0s 88us/step - loss: 0.5489 - accuracy: 0.7921 - val_loss: 0.5883 - val_accuracy: 0.7143
Epoch 59/1000
178/178 [==============================] - 0s 105us/step - loss: 0.5411 - accuracy: 0.7978 - val_loss: 0.5783 - val_accuracy: 0.7403
Epoch 60/1000
178/178 [==============================] - 0s 89us/step - loss: 0.5449 - accuracy: 0.7921 - val_loss: 0.5899 - val_accuracy: 0.7013
Epoch 61/1000
178/178 [==============================] - 0s 86us/step - loss: 0.5448 - accuracy: 0.7978 - val_loss: 0.5927 - val_accuracy: 0.7273
Epoch 62/1000
178/178 [==============================] - 0s 93us/step - loss: 0.5511 - accuracy: 0.7809 - val_loss: 0.5741 - val_accura

Epoch 112/1000
178/178 [==============================] - 0s 104us/step - loss: 0.4946 - accuracy: 0.7921 - val_loss: 0.5678 - val_accuracy: 0.7662
Epoch 113/1000
178/178 [==============================] - 0s 115us/step - loss: 0.5034 - accuracy: 0.7978 - val_loss: 0.5540 - val_accuracy: 0.7532
Epoch 114/1000
178/178 [==============================] - 0s 131us/step - loss: 0.4701 - accuracy: 0.8034 - val_loss: 0.7831 - val_accuracy: 0.6883
Epoch 115/1000
178/178 [==============================] - 0s 242us/step - loss: 0.5964 - accuracy: 0.7247 - val_loss: 0.5599 - val_accuracy: 0.7662
Epoch 116/1000
178/178 [==============================] - 0s 115us/step - loss: 0.4541 - accuracy: 0.8146 - val_loss: 0.5679 - val_accuracy: 0.7532
Epoch 117/1000
178/178 [==============================] - 0s 117us/step - loss: 0.4592 - accuracy: 0.8090 - val_loss: 0.5581 - val_accuracy: 0.7532
Epoch 118/1000
178/178 [==============================] - 0s 102us/step - loss: 0.4747 - accuracy: 0.7978 - val_

178/178 [==============================] - 0s 110us/step - loss: 0.4738 - accuracy: 0.8034 - val_loss: 0.6210 - val_accuracy: 0.7403
Epoch 168/1000
178/178 [==============================] - 0s 103us/step - loss: 0.4528 - accuracy: 0.8090 - val_loss: 0.5631 - val_accuracy: 0.7792
Epoch 169/1000
178/178 [==============================] - 0s 109us/step - loss: 0.4502 - accuracy: 0.7921 - val_loss: 0.5708 - val_accuracy: 0.7792
Epoch 170/1000
178/178 [==============================] - ETA: 0s - loss: 0.3893 - accuracy: 0.84 - 0s 109us/step - loss: 0.4198 - accuracy: 0.8202 - val_loss: 0.5653 - val_accuracy: 0.7662
Epoch 171/1000
178/178 [==============================] - 0s 104us/step - loss: 0.4913 - accuracy: 0.7753 - val_loss: 0.5794 - val_accuracy: 0.7792
Epoch 172/1000
178/178 [==============================] - 0s 88us/step - loss: 0.4562 - accuracy: 0.8090 - val_loss: 0.5885 - val_accuracy: 0.7792
Epoch 173/1000
178/178 [==============================] - 0s 99us/step - loss: 0.4289 

178/178 [==============================] - 0s 89us/step - loss: 0.4210 - accuracy: 0.8202 - val_loss: 0.6049 - val_accuracy: 0.7792
Epoch 223/1000
178/178 [==============================] - 0s 86us/step - loss: 0.4259 - accuracy: 0.8258 - val_loss: 0.5755 - val_accuracy: 0.7662
Epoch 224/1000
178/178 [==============================] - 0s 84us/step - loss: 0.4207 - accuracy: 0.8202 - val_loss: 0.5768 - val_accuracy: 0.7662
Epoch 225/1000
178/178 [==============================] - 0s 96us/step - loss: 0.4191 - accuracy: 0.8146 - val_loss: 0.5785 - val_accuracy: 0.7532
Epoch 226/1000
178/178 [==============================] - 0s 92us/step - loss: 0.4248 - accuracy: 0.8146 - val_loss: 0.6099 - val_accuracy: 0.7532
Epoch 227/1000
178/178 [==============================] - 0s 104us/step - loss: 0.4453 - accuracy: 0.8146 - val_loss: 0.5970 - val_accuracy: 0.7792
Epoch 228/1000
178/178 [==============================] - 0s 92us/step - loss: 0.4178 - accuracy: 0.8258 - val_loss: 0.5779 - val_ac

...

178/178 [==============================] - 0s 94us/step - loss: 0.3703 - accuracy: 0.8371 - val_loss: 0.6058 - val_accuracy: 0.7662
Epoch 773/1000
178/178 [==============================] - 0s 95us/step - loss: 0.3762 - accuracy: 0.8371 - val_loss: 0.6183 - val_accuracy: 0.7922
Epoch 774/1000
178/178 [==============================] - 0s 108us/step - loss: 0.3925 - accuracy: 0.8258 - val_loss: 0.6153 - val_accuracy: 0.7922
Epoch 775/1000
178/178 [==============================] - 0s 104us/step - loss: 0.3806 - accuracy: 0.8258 - val_loss: 0.6034 - val_accuracy: 0.7662
Epoch 776/1000
178/178 [==============================] - 0s 149us/step - loss: 0.3867 - accuracy: 0.8202 - val_loss: 0.6248 - val_accuracy: 0.7922
Epoch 777/1000
178/178 [==============================] - 0s 104us/step - loss: 0.3799 - accuracy: 0.8258 - val_loss: 0.6064 - val_accuracy: 0.7922
Epoch 778/1000
178/178 [==============================] - 0s 116us/step - loss: 0.3750 - accuracy: 0.8258 - val_loss: 0.6072 - va

178/178 [==============================] - 0s 92us/step - loss: 0.3625 - accuracy: 0.8371 - val_loss: 0.6376 - val_accuracy: 0.7922
Epoch 828/1000
178/178 [==============================] - 0s 104us/step - loss: 0.3783 - accuracy: 0.8202 - val_loss: 0.6293 - val_accuracy: 0.7922
Epoch 829/1000
178/178 [==============================] - 0s 115us/step - loss: 0.3676 - accuracy: 0.8315 - val_loss: 0.6055 - val_accuracy: 0.7922
Epoch 830/1000
178/178 [==============================] - 0s 109us/step - loss: 0.4042 - accuracy: 0.8146 - val_loss: 0.6159 - val_accuracy: 0.7922
Epoch 831/1000
178/178 [==============================] - 0s 98us/step - loss: 0.3714 - accuracy: 0.8371 - val_loss: 0.6232 - val_accuracy: 0.7922
Epoch 832/1000
178/178 [==============================] - 0s 97us/step - loss: 0.3786 - accuracy: 0.8258 - val_loss: 0.6097 - val_accuracy: 0.7792
Epoch 833/1000
178/178 [==============================] - 0s 105us/step - loss: 0.3746 - accuracy: 0.8258 - val_loss: 0.6075 - val

Epoch 883/1000
178/178 [==============================] - 0s 139us/step - loss: 0.3650 - accuracy: 0.8371 - val_loss: 0.6158 - val_accuracy: 0.7922
Epoch 884/1000
178/178 [==============================] - 0s 134us/step - loss: 0.3650 - accuracy: 0.8371 - val_loss: 0.6123 - val_accuracy: 0.7922
Epoch 885/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3624 - accuracy: 0.8315 - val_loss: 0.6082 - val_accuracy: 0.7922
Epoch 886/1000
178/178 [==============================] - 0s 101us/step - loss: 0.3632 - accuracy: 0.8258 - val_loss: 0.6110 - val_accuracy: 0.7792
Epoch 887/1000
178/178 [==============================] - 0s 100us/step - loss: 0.3715 - accuracy: 0.8258 - val_loss: 0.6258 - val_accuracy: 0.7922
Epoch 888/1000
178/178 [==============================] - 0s 166us/step - loss: 0.3649 - accuracy: 0.8315 - val_loss: 0.6131 - val_accuracy: 0.7792
Epoch 889/1000
178/178 [==============================] - 0s 115us/step - loss: 0.3644 - accuracy: 0.8371 - val_l

178/178 [==============================] - 0s 106us/step - loss: 0.3587 - accuracy: 0.8258 - val_loss: 0.6134 - val_accuracy: 0.7922
Epoch 939/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3653 - accuracy: 0.8315 - val_loss: 0.6283 - val_accuracy: 0.7922
Epoch 940/1000
178/178 [==============================] - 0s 111us/step - loss: 0.3772 - accuracy: 0.8258 - val_loss: 0.6372 - val_accuracy: 0.7922
Epoch 941/1000
178/178 [==============================] - 0s 103us/step - loss: 0.3666 - accuracy: 0.8315 - val_loss: 0.6168 - val_accuracy: 0.7922
Epoch 942/1000
178/178 [==============================] - 0s 117us/step - loss: 0.3684 - accuracy: 0.8202 - val_loss: 0.6146 - val_accuracy: 0.8052
Epoch 943/1000
178/178 [==============================] - 0s 101us/step - loss: 0.3604 - accuracy: 0.8371 - val_loss: 0.6400 - val_accuracy: 0.7922
Epoch 944/1000
178/178 [==============================] - 0s 104us/step - loss: 0.3653 - accuracy: 0.8258 - val_loss: 0.6127 - v

178/178 [==============================] - 0s 103us/step - loss: 0.3560 - accuracy: 0.8315 - val_loss: 0.6170 - val_accuracy: 0.7922
Epoch 994/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3599 - accuracy: 0.8371 - val_loss: 0.6116 - val_accuracy: 0.7922
Epoch 995/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3695 - accuracy: 0.8258 - val_loss: 0.6109 - val_accuracy: 0.7922
Epoch 996/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3565 - accuracy: 0.8371 - val_loss: 0.6122 - val_accuracy: 0.7922
Epoch 997/1000
178/178 [==============================] - 0s 96us/step - loss: 0.3577 - accuracy: 0.8315 - val_loss: 0.6167 - val_accuracy: 0.7792
Epoch 998/1000
178/178 [==============================] - 0s 104us/step - loss: 0.3740 - accuracy: 0.8427 - val_loss: 0.6093 - val_accuracy: 0.7792
Epoch 999/1000
178/178 [==============================] - 0s 102us/step - loss: 0.3677 - accuracy: 0.8315 - val_loss: 0.6248 - val_

In [122]:
acc_test2 = model2_2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 214us/step
test accuracy: 77.92%


In [123]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [124]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [125]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 136 samples, validate on 59 samples
Epoch 1/1000
136/136 [==============================] - 0s 730us/step - loss: 6.5434 - accuracy: 0.5294 - val_loss: 4.4103 - val_accuracy: 0.5085
Epoch 2/1000
136/136 [==============================] - 0s 110us/step - loss: 3.1932 - accuracy: 0.5515 - val_loss: 1.1796 - val_accuracy: 0.6610
Epoch 3/1000
136/136 [==============================] - 0s 126us/step - loss: 1.2345 - accuracy: 0.7206 - val_loss: 1.9106 - val_accuracy: 0.6102
Epoch 4/1000
136/136 [==============================] - 0s 194us/step - loss: 1.3830 - accuracy: 0.7574 - val_loss: 0.5922 - val_accuracy: 0.7627
Epoch 5/1000
136/136 [==============================] - 0s 151us/step - loss: 2.5971 - accuracy: 0.6618 - val_loss: 2.0414 - val_accuracy: 0.6949
Epoch 6/1000
136/136 [==============================] - 0s 138us/step - loss: 1.0489 - accuracy: 0.7353 - val_loss: 1.6082 - val_accuracy: 0.5932
Epoch 7/1000
136/136 [==============================] - 0s 158us/step - loss: 1

136/136 [==============================] - 0s 120us/step - loss: 0.3746 - accuracy: 0.9191 - val_loss: 0.4680 - val_accuracy: 0.7966
Epoch 57/1000
136/136 [==============================] - 0s 126us/step - loss: 0.3630 - accuracy: 0.8750 - val_loss: 0.4456 - val_accuracy: 0.8136
Epoch 58/1000
136/136 [==============================] - 0s 115us/step - loss: 0.3443 - accuracy: 0.9265 - val_loss: 1.6885 - val_accuracy: 0.7288
Epoch 59/1000
136/136 [==============================] - 0s 144us/step - loss: 0.5550 - accuracy: 0.8750 - val_loss: 0.4566 - val_accuracy: 0.8136
Epoch 60/1000
136/136 [==============================] - 0s 99us/step - loss: 0.3302 - accuracy: 0.9118 - val_loss: 0.5422 - val_accuracy: 0.6949
Epoch 61/1000
136/136 [==============================] - 0s 116us/step - loss: 0.3714 - accuracy: 0.8529 - val_loss: 0.5172 - val_accuracy: 0.7288
Epoch 62/1000
136/136 [==============================] - 0s 103us/step - loss: 0.4120 - accuracy: 0.8456 - val_loss: 0.4448 - val_acc

Epoch 112/1000
136/136 [==============================] - ETA: 0s - loss: 0.2900 - accuracy: 0.90 - 0s 137us/step - loss: 0.2705 - accuracy: 0.9191 - val_loss: 0.6109 - val_accuracy: 0.7458
Epoch 113/1000
136/136 [==============================] - 0s 134us/step - loss: 0.2594 - accuracy: 0.9118 - val_loss: 0.4485 - val_accuracy: 0.7966
Epoch 114/1000
136/136 [==============================] - 0s 147us/step - loss: 0.2614 - accuracy: 0.9191 - val_loss: 0.4506 - val_accuracy: 0.8983
Epoch 115/1000
136/136 [==============================] - 0s 150us/step - loss: 0.2408 - accuracy: 0.9485 - val_loss: 0.5677 - val_accuracy: 0.7458
Epoch 116/1000
136/136 [==============================] - 0s 149us/step - loss: 0.2655 - accuracy: 0.9044 - val_loss: 0.4493 - val_accuracy: 0.8475
Epoch 117/1000
136/136 [==============================] - 0s 128us/step - loss: 0.2354 - accuracy: 0.9632 - val_loss: 0.4171 - val_accuracy: 0.8814
Epoch 118/1000
136/136 [==============================] - 0s 119us/ste

Epoch 167/1000
136/136 [==============================] - 0s 103us/step - loss: 0.2192 - accuracy: 0.8971 - val_loss: 0.4106 - val_accuracy: 0.8136
Epoch 168/1000
136/136 [==============================] - 0s 116us/step - loss: 0.2079 - accuracy: 0.9118 - val_loss: 0.4314 - val_accuracy: 0.9153
Epoch 169/1000
136/136 [==============================] - 0s 111us/step - loss: 0.2090 - accuracy: 0.9191 - val_loss: 0.6478 - val_accuracy: 0.7458
Epoch 170/1000
136/136 [==============================] - 0s 109us/step - loss: 0.2564 - accuracy: 0.9191 - val_loss: 0.6159 - val_accuracy: 0.7458
Epoch 171/1000
136/136 [==============================] - 0s 174us/step - loss: 0.2707 - accuracy: 0.8824 - val_loss: 0.4460 - val_accuracy: 0.8983
Epoch 172/1000
136/136 [==============================] - 0s 118us/step - loss: 0.2052 - accuracy: 0.9853 - val_loss: 0.4251 - val_accuracy: 0.8814
Epoch 173/1000
136/136 [==============================] - 0s 130us/step - loss: 0.2086 - accuracy: 0.9412 - val_

136/136 [==============================] - 0s 128us/step - loss: 0.2461 - accuracy: 0.9044 - val_loss: 0.3853 - val_accuracy: 0.8644
Epoch 223/1000
136/136 [==============================] - 0s 121us/step - loss: 0.2069 - accuracy: 0.9485 - val_loss: 1.1056 - val_accuracy: 0.7119
Epoch 224/1000
136/136 [==============================] - 0s 146us/step - loss: 0.3487 - accuracy: 0.8971 - val_loss: 0.4181 - val_accuracy: 0.7288
Epoch 225/1000
136/136 [==============================] - 0s 113us/step - loss: 0.2095 - accuracy: 0.8971 - val_loss: 0.4112 - val_accuracy: 0.7458
Epoch 226/1000
136/136 [==============================] - 0s 118us/step - loss: 0.2080 - accuracy: 0.8971 - val_loss: 0.4103 - val_accuracy: 0.8136
Epoch 227/1000
136/136 [==============================] - 0s 112us/step - loss: 0.2103 - accuracy: 0.9191 - val_loss: 0.3889 - val_accuracy: 0.8644
Epoch 228/1000
136/136 [==============================] - 0s 124us/step - loss: 0.2149 - accuracy: 0.9706 - val_loss: 0.4041 - 

...

136/136 [==============================] - 0s 97us/step - loss: 0.1924 - accuracy: 0.8971 - val_loss: 0.3956 - val_accuracy: 0.7458
Epoch 773/1000
136/136 [==============================] - 0s 111us/step - loss: 0.1726 - accuracy: 0.8971 - val_loss: 0.3917 - val_accuracy: 0.7458
Epoch 774/1000
136/136 [==============================] - 0s 113us/step - loss: 0.1714 - accuracy: 0.8971 - val_loss: 0.3910 - val_accuracy: 0.7458
Epoch 775/1000
136/136 [==============================] - ETA: 0s - loss: 0.1582 - accuracy: 0.90 - 0s 110us/step - loss: 0.1882 - accuracy: 0.8971 - val_loss: 0.4326 - val_accuracy: 0.7119
Epoch 776/1000
136/136 [==============================] - 0s 118us/step - loss: 0.1858 - accuracy: 0.8971 - val_loss: 0.4095 - val_accuracy: 0.7288
Epoch 777/1000
136/136 [==============================] - 0s 112us/step - loss: 0.1698 - accuracy: 0.8971 - val_loss: 0.3985 - val_accuracy: 0.7288
Epoch 778/1000
136/136 [==============================] - 0s 105us/step - loss: 0.1675

136/136 [==============================] - 0s 124us/step - loss: 0.1623 - accuracy: 0.8971 - val_loss: 0.4021 - val_accuracy: 0.7458
Epoch 828/1000
136/136 [==============================] - 0s 121us/step - loss: 0.1624 - accuracy: 0.9044 - val_loss: 0.3927 - val_accuracy: 0.7627
Epoch 829/1000
136/136 [==============================] - 0s 116us/step - loss: 0.1659 - accuracy: 0.8971 - val_loss: 0.5097 - val_accuracy: 0.7119
Epoch 830/1000
136/136 [==============================] - 0s 116us/step - loss: 0.1757 - accuracy: 0.8897 - val_loss: 0.4034 - val_accuracy: 0.7458
Epoch 831/1000
136/136 [==============================] - 0s 108us/step - loss: 0.1600 - accuracy: 0.9044 - val_loss: 0.3983 - val_accuracy: 0.7458
Epoch 832/1000
136/136 [==============================] - 0s 105us/step - loss: 0.1609 - accuracy: 0.8971 - val_loss: 0.4006 - val_accuracy: 0.7627
Epoch 833/1000
136/136 [==============================] - 0s 107us/step - loss: 0.1654 - accuracy: 0.9118 - val_loss: 0.4352 - 

136/136 [==============================] - 0s 105us/step - loss: 0.2114 - accuracy: 0.8676 - val_loss: 0.4089 - val_accuracy: 0.7288
Epoch 883/1000
136/136 [==============================] - 0s 119us/step - loss: 0.1646 - accuracy: 0.9044 - val_loss: 0.4099 - val_accuracy: 0.7288
Epoch 884/1000
136/136 [==============================] - 0s 120us/step - loss: 0.1641 - accuracy: 0.9044 - val_loss: 0.4210 - val_accuracy: 0.7288
Epoch 885/1000
136/136 [==============================] - 0s 112us/step - loss: 0.1672 - accuracy: 0.8971 - val_loss: 0.4085 - val_accuracy: 0.7458
Epoch 886/1000
136/136 [==============================] - 0s 110us/step - loss: 0.1618 - accuracy: 0.9118 - val_loss: 0.4116 - val_accuracy: 0.7288
Epoch 887/1000
136/136 [==============================] - 0s 108us/step - loss: 0.1684 - accuracy: 0.8971 - val_loss: 0.4099 - val_accuracy: 0.7458
Epoch 888/1000
136/136 [==============================] - 0s 116us/step - loss: 0.1664 - accuracy: 0.8971 - val_loss: 0.4116 - 

136/136 [==============================] - 0s 97us/step - loss: 0.1625 - accuracy: 0.9044 - val_loss: 0.4201 - val_accuracy: 0.7119
Epoch 938/1000
136/136 [==============================] - 0s 101us/step - loss: 0.1623 - accuracy: 0.9044 - val_loss: 0.4175 - val_accuracy: 0.7288
Epoch 939/1000
136/136 [==============================] - 0s 126us/step - loss: 0.1573 - accuracy: 0.9044 - val_loss: 0.4216 - val_accuracy: 0.7288
Epoch 940/1000
136/136 [==============================] - 0s 101us/step - loss: 0.1621 - accuracy: 0.9044 - val_loss: 0.4163 - val_accuracy: 0.7288
Epoch 941/1000
136/136 [==============================] - 0s 130us/step - loss: 0.1573 - accuracy: 0.9044 - val_loss: 0.4849 - val_accuracy: 0.7288
Epoch 942/1000
136/136 [==============================] - 0s 120us/step - loss: 0.1677 - accuracy: 0.8971 - val_loss: 0.4147 - val_accuracy: 0.7288
Epoch 943/1000
136/136 [==============================] - 0s 119us/step - loss: 0.1555 - accuracy: 0.9118 - val_loss: 0.4120 - v

136/136 [==============================] - 0s 86us/step - loss: 0.1838 - accuracy: 0.8971 - val_loss: 0.4013 - val_accuracy: 0.7458
Epoch 993/1000
136/136 [==============================] - 0s 91us/step - loss: 0.1721 - accuracy: 0.8971 - val_loss: 0.4076 - val_accuracy: 0.7458
Epoch 994/1000
136/136 [==============================] - 0s 85us/step - loss: 0.1669 - accuracy: 0.8971 - val_loss: 0.4035 - val_accuracy: 0.7458
Epoch 995/1000
136/136 [==============================] - 0s 106us/step - loss: 0.1647 - accuracy: 0.8971 - val_loss: 0.3909 - val_accuracy: 0.7458
Epoch 996/1000
136/136 [==============================] - 0s 96us/step - loss: 0.1524 - accuracy: 0.8971 - val_loss: 0.3981 - val_accuracy: 0.7627
Epoch 997/1000
136/136 [==============================] - 0s 115us/step - loss: 0.1642 - accuracy: 0.8971 - val_loss: 0.3943 - val_accuracy: 0.7458
Epoch 998/1000
136/136 [==============================] - 0s 104us/step - loss: 0.1569 - accuracy: 0.9044 - val_loss: 0.3959 - val_

In [126]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


59/59 [==============================] - 0s 106us/step
combination test accuracy: 72.88%


In [139]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [140]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [141]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 136 samples, validate on 59 samples
Epoch 1/1000
136/136 [==============================] - 0s 1ms/step - loss: 4.0522 - accuracy: 0.3897 - val_loss: 2.6747 - val_accuracy: 0.4407
Epoch 2/1000
136/136 [==============================] - 0s 174us/step - loss: 4.2969 - accuracy: 0.3529 - val_loss: 2.5276 - val_accuracy: 0.4576
Epoch 3/1000
136/136 [==============================] - 0s 238us/step - loss: 3.7623 - accuracy: 0.4485 - val_loss: 1.8259 - val_accuracy: 0.4576
Epoch 4/1000
136/136 [==============================] - 0s 239us/step - loss: 3.4687 - accuracy: 0.5074 - val_loss: 1.0943 - val_accuracy: 0.4576
Epoch 5/1000
136/136 [==============================] - 0s 226us/step - loss: 2.2506 - accuracy: 0.5441 - val_loss: 0.7394 - val_accuracy: 0.5424
Epoch 6/1000
136/136 [==============================] - 0s 219us/step - loss: 1.6836 - accuracy: 0.5588 - val_loss: 0.7229 - val_accuracy: 0.5424
Epoch 7/1000
136/136 [==============================] - 0s 246us/step - loss: 2.2

136/136 [==============================] - 0s 283us/step - loss: 1.5046 - accuracy: 0.8088 - val_loss: 0.6314 - val_accuracy: 0.5763
Epoch 57/1000
136/136 [==============================] - 0s 292us/step - loss: 1.8111 - accuracy: 0.8088 - val_loss: 0.6318 - val_accuracy: 0.5763
Epoch 58/1000
136/136 [==============================] - 0s 246us/step - loss: 0.9373 - accuracy: 0.8603 - val_loss: 0.6343 - val_accuracy: 0.5932
Epoch 59/1000
136/136 [==============================] - 0s 221us/step - loss: 2.1457 - accuracy: 0.7721 - val_loss: 0.6348 - val_accuracy: 0.5932
Epoch 60/1000
136/136 [==============================] - 0s 197us/step - loss: 1.7068 - accuracy: 0.7941 - val_loss: 0.6148 - val_accuracy: 0.5932
Epoch 61/1000
136/136 [==============================] - 0s 214us/step - loss: 1.4694 - accuracy: 0.8235 - val_loss: 0.6262 - val_accuracy: 0.6441
Epoch 62/1000
136/136 [==============================] - 0s 244us/step - loss: 1.5818 - accuracy: 0.8015 - val_loss: 0.6219 - val_ac

Epoch 112/1000
136/136 [==============================] - 0s 194us/step - loss: 2.0772 - accuracy: 0.7868 - val_loss: 0.5910 - val_accuracy: 0.6780
Epoch 113/1000
136/136 [==============================] - 0s 217us/step - loss: 1.6487 - accuracy: 0.8088 - val_loss: 0.6011 - val_accuracy: 0.6780
Epoch 114/1000
136/136 [==============================] - 0s 211us/step - loss: 1.3257 - accuracy: 0.8162 - val_loss: 0.6020 - val_accuracy: 0.6780
Epoch 115/1000
136/136 [==============================] - 0s 199us/step - loss: 1.6337 - accuracy: 0.8088 - val_loss: 0.5936 - val_accuracy: 0.6780
Epoch 116/1000
136/136 [==============================] - 0s 193us/step - loss: 0.7390 - accuracy: 0.8750 - val_loss: 0.5983 - val_accuracy: 0.6780
Epoch 117/1000
136/136 [==============================] - 0s 189us/step - loss: 1.4187 - accuracy: 0.8162 - val_loss: 0.5928 - val_accuracy: 0.6780
Epoch 118/1000
136/136 [==============================] - 0s 196us/step - loss: 1.3934 - accuracy: 0.8382 - val_

136/136 [==============================] - 0s 194us/step - loss: 1.4041 - accuracy: 0.8309 - val_loss: 0.6073 - val_accuracy: 0.6949
Epoch 168/1000
136/136 [==============================] - 0s 205us/step - loss: 1.0559 - accuracy: 0.8603 - val_loss: 0.5911 - val_accuracy: 0.6949
Epoch 169/1000
136/136 [==============================] - 0s 205us/step - loss: 1.4774 - accuracy: 0.8456 - val_loss: 0.5994 - val_accuracy: 0.6949
Epoch 170/1000
136/136 [==============================] - 0s 198us/step - loss: 1.6109 - accuracy: 0.8235 - val_loss: 0.5968 - val_accuracy: 0.6949
Epoch 171/1000
136/136 [==============================] - 0s 200us/step - loss: 1.6265 - accuracy: 0.8162 - val_loss: 0.5948 - val_accuracy: 0.6949
Epoch 172/1000
136/136 [==============================] - 0s 193us/step - loss: 1.0379 - accuracy: 0.8676 - val_loss: 0.5910 - val_accuracy: 0.6949
Epoch 173/1000
136/136 [==============================] - 0s 167us/step - loss: 0.8180 - accuracy: 0.8824 - val_loss: 0.5878 - 

136/136 [==============================] - 0s 209us/step - loss: 1.6968 - accuracy: 0.8309 - val_loss: 0.6027 - val_accuracy: 0.7119
Epoch 223/1000
136/136 [==============================] - 0s 193us/step - loss: 1.8408 - accuracy: 0.8015 - val_loss: 0.5963 - val_accuracy: 0.7119
Epoch 224/1000
136/136 [==============================] - 0s 177us/step - loss: 2.0565 - accuracy: 0.7941 - val_loss: 0.5926 - val_accuracy: 0.7119
Epoch 225/1000
136/136 [==============================] - 0s 204us/step - loss: 1.5807 - accuracy: 0.8382 - val_loss: 0.5960 - val_accuracy: 0.7119
Epoch 226/1000
136/136 [==============================] - 0s 203us/step - loss: 2.2750 - accuracy: 0.7794 - val_loss: 0.5860 - val_accuracy: 0.7119
Epoch 227/1000
136/136 [==============================] - 0s 196us/step - loss: 1.3680 - accuracy: 0.8456 - val_loss: 0.5884 - val_accuracy: 0.7119
Epoch 228/1000
136/136 [==============================] - 0s 209us/step - loss: 1.8509 - accuracy: 0.7941 - val_loss: 0.5924 - 

...

136/136 [==============================] - 0s 175us/step - loss: 1.9386 - accuracy: 0.8015 - val_loss: 0.6747 - val_accuracy: 0.7288
Epoch 773/1000
136/136 [==============================] - 0s 189us/step - loss: 1.7020 - accuracy: 0.8235 - val_loss: 0.6644 - val_accuracy: 0.7288
Epoch 774/1000
136/136 [==============================] - 0s 200us/step - loss: 1.4664 - accuracy: 0.8456 - val_loss: 0.6726 - val_accuracy: 0.7288
Epoch 775/1000
136/136 [==============================] - 0s 201us/step - loss: 1.7261 - accuracy: 0.8088 - val_loss: 0.6655 - val_accuracy: 0.7288
Epoch 776/1000
136/136 [==============================] - 0s 186us/step - loss: 0.9419 - accuracy: 0.8603 - val_loss: 0.6614 - val_accuracy: 0.7288
Epoch 777/1000
136/136 [==============================] - 0s 181us/step - loss: 2.0616 - accuracy: 0.7868 - val_loss: 0.6652 - val_accuracy: 0.7288
Epoch 778/1000
136/136 [==============================] - 0s 199us/step - loss: 2.5956 - accuracy: 0.7647 - val_loss: 0.6568 - 

136/136 [==============================] - 0s 191us/step - loss: 1.9359 - accuracy: 0.8015 - val_loss: 0.6670 - val_accuracy: 0.7288
Epoch 828/1000
136/136 [==============================] - 0s 209us/step - loss: 1.9146 - accuracy: 0.8162 - val_loss: 0.6714 - val_accuracy: 0.7288
Epoch 829/1000
136/136 [==============================] - 0s 224us/step - loss: 1.9246 - accuracy: 0.8088 - val_loss: 0.6599 - val_accuracy: 0.7288
Epoch 830/1000
136/136 [==============================] - 0s 201us/step - loss: 1.6011 - accuracy: 0.8235 - val_loss: 0.6645 - val_accuracy: 0.7288
Epoch 831/1000
136/136 [==============================] - 0s 199us/step - loss: 1.3413 - accuracy: 0.8603 - val_loss: 0.6706 - val_accuracy: 0.7288
Epoch 832/1000
136/136 [==============================] - 0s 248us/step - loss: 1.6898 - accuracy: 0.8309 - val_loss: 0.6578 - val_accuracy: 0.7288
Epoch 833/1000
136/136 [==============================] - 0s 198us/step - loss: 2.3962 - accuracy: 0.7647 - val_loss: 0.6630 - 

136/136 [==============================] - 0s 241us/step - loss: 1.6758 - accuracy: 0.8382 - val_loss: 0.6614 - val_accuracy: 0.7288
Epoch 883/1000
136/136 [==============================] - 0s 175us/step - loss: 1.5635 - accuracy: 0.8456 - val_loss: 0.6658 - val_accuracy: 0.7288
Epoch 884/1000
136/136 [==============================] - 0s 202us/step - loss: 1.6991 - accuracy: 0.8235 - val_loss: 0.6684 - val_accuracy: 0.7288
Epoch 885/1000
136/136 [==============================] - 0s 167us/step - loss: 1.0412 - accuracy: 0.8603 - val_loss: 0.6620 - val_accuracy: 0.7288
Epoch 886/1000
136/136 [==============================] - 0s 211us/step - loss: 1.2272 - accuracy: 0.8676 - val_loss: 0.6662 - val_accuracy: 0.7288
Epoch 887/1000
136/136 [==============================] - 0s 180us/step - loss: 1.8480 - accuracy: 0.7941 - val_loss: 0.6662 - val_accuracy: 0.7288
Epoch 888/1000
136/136 [==============================] - 0s 172us/step - loss: 1.7999 - accuracy: 0.8235 - val_loss: 0.6735 - 

Epoch 937/1000
136/136 [==============================] - 0s 232us/step - loss: 1.6105 - accuracy: 0.8162 - val_loss: 0.6648 - val_accuracy: 0.7288
Epoch 938/1000
136/136 [==============================] - 0s 187us/step - loss: 1.2402 - accuracy: 0.8603 - val_loss: 0.6671 - val_accuracy: 0.7288
Epoch 939/1000
136/136 [==============================] - 0s 198us/step - loss: 1.3870 - accuracy: 0.8309 - val_loss: 0.6656 - val_accuracy: 0.7288
Epoch 940/1000
136/136 [==============================] - 0s 200us/step - loss: 1.6863 - accuracy: 0.8309 - val_loss: 0.6564 - val_accuracy: 0.7288
Epoch 941/1000
136/136 [==============================] - 0s 196us/step - loss: 1.0155 - accuracy: 0.8750 - val_loss: 0.6610 - val_accuracy: 0.7288
Epoch 942/1000
136/136 [==============================] - 0s 184us/step - loss: 1.2864 - accuracy: 0.8309 - val_loss: 0.6619 - val_accuracy: 0.7288
Epoch 943/1000
136/136 [==============================] - 0s 200us/step - loss: 1.5982 - accuracy: 0.8235 - val_

136/136 [==============================] - 0s 177us/step - loss: 2.4935 - accuracy: 0.7647 - val_loss: 0.6635 - val_accuracy: 0.7288
Epoch 993/1000
136/136 [==============================] - 0s 203us/step - loss: 1.3504 - accuracy: 0.8529 - val_loss: 0.6673 - val_accuracy: 0.7288
Epoch 994/1000
136/136 [==============================] - 0s 192us/step - loss: 2.1456 - accuracy: 0.7941 - val_loss: 0.6676 - val_accuracy: 0.7288
Epoch 995/1000
136/136 [==============================] - 0s 199us/step - loss: 1.8445 - accuracy: 0.7941 - val_loss: 0.6705 - val_accuracy: 0.7288
Epoch 996/1000
136/136 [==============================] - 0s 196us/step - loss: 2.1688 - accuracy: 0.7794 - val_loss: 0.6706 - val_accuracy: 0.7288
Epoch 997/1000
136/136 [==============================] - 0s 230us/step - loss: 1.0380 - accuracy: 0.8603 - val_loss: 0.6711 - val_accuracy: 0.7288
Epoch 998/1000
136/136 [==============================] - 0s 206us/step - loss: 2.0445 - accuracy: 0.7941 - val_loss: 0.6461 - 

In [142]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

59/59 [==============================] - 0s 100us/step
combination test accuracy: 72.88%


In [143]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [144]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [145]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 488us/step - loss: 24.4062 - accuracy: 0.4719 - val_loss: 0.7352 - val_accuracy: 0.4848
Epoch 2/1000
231/231 [==============================] - 0s 96us/step - loss: 0.7544 - accuracy: 0.4892 - val_loss: 0.6915 - val_accuracy: 0.5556
Epoch 3/1000
231/231 [==============================] - 0s 130us/step - loss: 0.7969 - accuracy: 0.5195 - val_loss: 0.6773 - val_accuracy: 0.5657
Epoch 4/1000
231/231 [==============================] - 0s 101us/step - loss: 0.6858 - accuracy: 0.5065 - val_loss: 0.6735 - val_accuracy: 0.5152
Epoch 5/1000
231/231 [==============================] - 0s 115us/step - loss: 0.6860 - accuracy: 0.5195 - val_loss: 0.6700 - val_accuracy: 0.5455
Epoch 6/1000
231/231 [==============================] - 0s 116us/step - loss: 0.6859 - accuracy: 0.5022 - val_loss: 0.6669 - val_accuracy: 0.5960
Epoch 7/1000
231/231 [==============================] - 0s 126us/step - loss: 0

231/231 [==============================] - 0s 115us/step - loss: 0.7339 - accuracy: 0.6883 - val_loss: 0.6053 - val_accuracy: 0.7576
Epoch 57/1000
231/231 [==============================] - 0s 112us/step - loss: 0.6125 - accuracy: 0.7316 - val_loss: 0.6669 - val_accuracy: 0.6364
Epoch 58/1000
231/231 [==============================] - 0s 108us/step - loss: 0.6191 - accuracy: 0.7100 - val_loss: 0.6024 - val_accuracy: 0.7576
Epoch 59/1000
231/231 [==============================] - 0s 106us/step - loss: 0.6239 - accuracy: 0.7446 - val_loss: 0.6071 - val_accuracy: 0.7576
Epoch 60/1000
231/231 [==============================] - ETA: 0s - loss: 0.6378 - accuracy: 0.75 - 0s 95us/step - loss: 0.6189 - accuracy: 0.7316 - val_loss: 0.5995 - val_accuracy: 0.7576
Epoch 61/1000
231/231 [==============================] - 0s 101us/step - loss: 0.5959 - accuracy: 0.7316 - val_loss: 0.5965 - val_accuracy: 0.7576
Epoch 62/1000
231/231 [==============================] - 0s 97us/step - loss: 0.6302 - accu

Epoch 112/1000
231/231 [==============================] - 0s 84us/step - loss: 0.7753 - accuracy: 0.7186 - val_loss: 0.5570 - val_accuracy: 0.7475
Epoch 113/1000
231/231 [==============================] - 0s 105us/step - loss: 0.5773 - accuracy: 0.7186 - val_loss: 0.5522 - val_accuracy: 0.7677
Epoch 114/1000
231/231 [==============================] - 0s 98us/step - loss: 0.5730 - accuracy: 0.7273 - val_loss: 0.6312 - val_accuracy: 0.7475
Epoch 115/1000
231/231 [==============================] - 0s 93us/step - loss: 0.6014 - accuracy: 0.7316 - val_loss: 0.5661 - val_accuracy: 0.7576
Epoch 116/1000
231/231 [==============================] - 0s 91us/step - loss: 0.5625 - accuracy: 0.7403 - val_loss: 0.5509 - val_accuracy: 0.7677
Epoch 117/1000
231/231 [==============================] - 0s 92us/step - loss: 0.5701 - accuracy: 0.7100 - val_loss: 0.6312 - val_accuracy: 0.6869
Epoch 118/1000
231/231 [==============================] - 0s 87us/step - loss: 0.5820 - accuracy: 0.7273 - val_loss: 

231/231 [==============================] - 0s 90us/step - loss: 0.5760 - accuracy: 0.7229 - val_loss: 0.6782 - val_accuracy: 0.6970
Epoch 168/1000
231/231 [==============================] - 0s 94us/step - loss: 0.5702 - accuracy: 0.7359 - val_loss: 0.5304 - val_accuracy: 0.7778
Epoch 169/1000
231/231 [==============================] - 0s 96us/step - loss: 0.5449 - accuracy: 0.7403 - val_loss: 0.5262 - val_accuracy: 0.7778
Epoch 170/1000
231/231 [==============================] - 0s 100us/step - loss: 0.5336 - accuracy: 0.7403 - val_loss: 0.5274 - val_accuracy: 0.7778
Epoch 171/1000
231/231 [==============================] - 0s 96us/step - loss: 0.5165 - accuracy: 0.7489 - val_loss: 0.5256 - val_accuracy: 0.7778
Epoch 172/1000
231/231 [==============================] - 0s 105us/step - loss: 0.5731 - accuracy: 0.7100 - val_loss: 0.5611 - val_accuracy: 0.7576
Epoch 173/1000
231/231 [==============================] - 0s 97us/step - loss: 0.5587 - accuracy: 0.7316 - val_loss: 0.5273 - val_a

Epoch 223/1000
231/231 [==============================] - 0s 77us/step - loss: 0.5216 - accuracy: 0.7403 - val_loss: 0.5272 - val_accuracy: 0.7576
Epoch 224/1000
231/231 [==============================] - 0s 92us/step - loss: 0.5738 - accuracy: 0.7316 - val_loss: 0.5126 - val_accuracy: 0.7677
Epoch 225/1000
231/231 [==============================] - 0s 126us/step - loss: 0.5319 - accuracy: 0.7403 - val_loss: 0.5160 - val_accuracy: 0.7677
Epoch 226/1000
231/231 [==============================] - 0s 89us/step - loss: 0.5118 - accuracy: 0.7403 - val_loss: 0.5158 - val_accuracy: 0.7576
Epoch 227/1000
231/231 [==============================] - 0s 89us/step - loss: 0.5889 - accuracy: 0.7316 - val_loss: 0.9684 - val_accuracy: 0.6162
Epoch 228/1000
231/231 [==============================] - 0s 94us/step - loss: 0.6303 - accuracy: 0.7056 - val_loss: 0.7851 - val_accuracy: 0.6465
Epoch 229/1000
231/231 [==============================] - 0s 96us/step - loss: 0.6050 - accuracy: 0.7403 - val_loss: 

...

231/231 [==============================] - 0s 116us/step - loss: 0.4571 - accuracy: 0.7619 - val_loss: 0.5599 - val_accuracy: 0.7374
Epoch 778/1000
231/231 [==============================] - 0s 114us/step - loss: 0.4970 - accuracy: 0.7186 - val_loss: 0.5430 - val_accuracy: 0.7576
Epoch 779/1000
231/231 [==============================] - 0s 133us/step - loss: 0.4881 - accuracy: 0.7489 - val_loss: 0.5336 - val_accuracy: 0.7778
Epoch 780/1000
231/231 [==============================] - 0s 146us/step - loss: 0.4828 - accuracy: 0.7619 - val_loss: 0.5278 - val_accuracy: 0.7778
Epoch 781/1000
231/231 [==============================] - 0s 167us/step - loss: 0.4782 - accuracy: 0.7576 - val_loss: 0.5264 - val_accuracy: 0.7778
Epoch 782/1000
231/231 [==============================] - 0s 118us/step - loss: 0.4762 - accuracy: 0.7532 - val_loss: 0.5247 - val_accuracy: 0.7677
Epoch 783/1000
231/231 [==============================] - 0s 110us/step - loss: 0.4742 - accuracy: 0.7662 - val_loss: 0.5251 - 

Epoch 833/1000
231/231 [==============================] - 0s 87us/step - loss: 0.4500 - accuracy: 0.7662 - val_loss: 0.4864 - val_accuracy: 0.7677
Epoch 834/1000
231/231 [==============================] - 0s 98us/step - loss: 0.4523 - accuracy: 0.7706 - val_loss: 0.5015 - val_accuracy: 0.7677
Epoch 835/1000
231/231 [==============================] - 0s 100us/step - loss: 0.4553 - accuracy: 0.7662 - val_loss: 0.4851 - val_accuracy: 0.7778
Epoch 836/1000
231/231 [==============================] - 0s 106us/step - loss: 0.4611 - accuracy: 0.7662 - val_loss: 0.5069 - val_accuracy: 0.7677
Epoch 837/1000
231/231 [==============================] - 0s 89us/step - loss: 0.4528 - accuracy: 0.7619 - val_loss: 0.4862 - val_accuracy: 0.7778
Epoch 838/1000
231/231 [==============================] - 0s 101us/step - loss: 0.4480 - accuracy: 0.7662 - val_loss: 0.4891 - val_accuracy: 0.7677
Epoch 839/1000
231/231 [==============================] - 0s 111us/step - loss: 0.4512 - accuracy: 0.7706 - val_los

231/231 [==============================] - 0s 90us/step - loss: 0.4443 - accuracy: 0.7706 - val_loss: 0.4857 - val_accuracy: 0.7778
Epoch 889/1000
231/231 [==============================] - 0s 94us/step - loss: 0.4443 - accuracy: 0.7706 - val_loss: 0.5214 - val_accuracy: 0.7273
Epoch 890/1000
231/231 [==============================] - 0s 97us/step - loss: 0.4495 - accuracy: 0.7662 - val_loss: 0.4855 - val_accuracy: 0.7778
Epoch 891/1000
231/231 [==============================] - 0s 91us/step - loss: 0.4632 - accuracy: 0.7576 - val_loss: 0.4949 - val_accuracy: 0.7677
Epoch 892/1000
231/231 [==============================] - 0s 95us/step - loss: 0.4433 - accuracy: 0.7749 - val_loss: 0.4846 - val_accuracy: 0.7778
Epoch 893/1000
231/231 [==============================] - 0s 89us/step - loss: 0.4427 - accuracy: 0.7792 - val_loss: 0.7407 - val_accuracy: 0.7273
Epoch 894/1000
231/231 [==============================] - 0s 91us/step - loss: 0.5190 - accuracy: 0.7749 - val_loss: 0.4841 - val_acc

Epoch 944/1000
231/231 [==============================] - 0s 103us/step - loss: 0.4385 - accuracy: 0.7706 - val_loss: 0.4866 - val_accuracy: 0.7778
Epoch 945/1000
231/231 [==============================] - 0s 100us/step - loss: 0.4406 - accuracy: 0.7792 - val_loss: 0.4871 - val_accuracy: 0.7778
Epoch 946/1000
231/231 [==============================] - 0s 100us/step - loss: 0.4392 - accuracy: 0.7749 - val_loss: 0.4913 - val_accuracy: 0.7879
Epoch 947/1000
231/231 [==============================] - 0s 86us/step - loss: 0.4461 - accuracy: 0.7749 - val_loss: 0.5086 - val_accuracy: 0.7677
Epoch 948/1000
231/231 [==============================] - 0s 95us/step - loss: 0.4467 - accuracy: 0.7706 - val_loss: 0.4893 - val_accuracy: 0.7778
Epoch 949/1000
231/231 [==============================] - 0s 117us/step - loss: 0.4390 - accuracy: 0.7792 - val_loss: 0.5293 - val_accuracy: 0.7273
Epoch 950/1000
231/231 [==============================] - 0s 98us/step - loss: 0.4532 - accuracy: 0.7792 - val_los

231/231 [==============================] - 0s 107us/step - loss: 0.4372 - accuracy: 0.7749 - val_loss: 0.4852 - val_accuracy: 0.7778
Epoch 1000/1000
231/231 [==============================] - 0s 125us/step - loss: 0.4401 - accuracy: 0.7706 - val_loss: 0.5253 - val_accuracy: 0.7576


In [146]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

99/99 [==============================] - 0s 122us/step
combination test accuracy: 75.76%


In [159]:
#### improve neural network on over-sampling data
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [160]:
model2_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [161]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 231 samples, validate on 99 samples
Epoch 1/1000
231/231 [==============================] - 0s 549us/step - loss: 6.5112 - accuracy: 0.4848 - val_loss: 0.8533 - val_accuracy: 0.4949
Epoch 2/1000
231/231 [==============================] - 0s 146us/step - loss: 0.6989 - accuracy: 0.5368 - val_loss: 0.6912 - val_accuracy: 0.5556
Epoch 3/1000
231/231 [==============================] - 0s 186us/step - loss: 0.6911 - accuracy: 0.5671 - val_loss: 0.6801 - val_accuracy: 0.5657
Epoch 4/1000
231/231 [==============================] - 0s 170us/step - loss: 0.6742 - accuracy: 0.5931 - val_loss: 0.6688 - val_accuracy: 0.6768
Epoch 5/1000
231/231 [==============================] - 0s 203us/step - loss: 0.6693 - accuracy: 0.6537 - val_loss: 0.6638 - val_accuracy: 0.6768
Epoch 6/1000
231/231 [==============================] - 0s 170us/step - loss: 0.6637 - accuracy: 0.6494 - val_loss: 0.6641 - val_accuracy: 0.6162
Epoch 7/1000
231/231 [==============================] - 0s 164us/step - loss: 0

231/231 [==============================] - 0s 186us/step - loss: 0.5294 - accuracy: 0.7359 - val_loss: 0.5236 - val_accuracy: 0.7879
Epoch 57/1000
231/231 [==============================] - 0s 178us/step - loss: 0.5412 - accuracy: 0.7056 - val_loss: 0.5449 - val_accuracy: 0.7475
Epoch 58/1000
231/231 [==============================] - 0s 158us/step - loss: 0.5250 - accuracy: 0.7316 - val_loss: 0.5400 - val_accuracy: 0.7778
Epoch 59/1000
231/231 [==============================] - 0s 139us/step - loss: 0.5222 - accuracy: 0.7273 - val_loss: 0.5323 - val_accuracy: 0.7374
Epoch 60/1000
231/231 [==============================] - 0s 137us/step - loss: 0.5197 - accuracy: 0.7359 - val_loss: 0.5458 - val_accuracy: 0.7273
Epoch 61/1000
231/231 [==============================] - 0s 139us/step - loss: 0.5233 - accuracy: 0.7403 - val_loss: 0.5289 - val_accuracy: 0.7879
Epoch 62/1000
231/231 [==============================] - 0s 141us/step - loss: 0.5421 - accuracy: 0.6970 - val_loss: 0.5346 - val_ac

Epoch 112/1000
231/231 [==============================] - 0s 155us/step - loss: 0.4935 - accuracy: 0.7489 - val_loss: 0.5222 - val_accuracy: 0.7273
Epoch 113/1000
231/231 [==============================] - 0s 157us/step - loss: 0.5251 - accuracy: 0.7143 - val_loss: 0.5303 - val_accuracy: 0.7475
Epoch 114/1000
231/231 [==============================] - 0s 138us/step - loss: 0.5059 - accuracy: 0.7273 - val_loss: 0.5152 - val_accuracy: 0.7475
Epoch 115/1000
231/231 [==============================] - 0s 133us/step - loss: 0.5004 - accuracy: 0.7532 - val_loss: 0.5060 - val_accuracy: 0.7576
Epoch 116/1000
231/231 [==============================] - 0s 139us/step - loss: 0.4955 - accuracy: 0.7359 - val_loss: 0.5237 - val_accuracy: 0.7475
Epoch 117/1000
231/231 [==============================] - 0s 146us/step - loss: 0.4880 - accuracy: 0.7662 - val_loss: 0.5045 - val_accuracy: 0.7475
Epoch 118/1000
231/231 [==============================] - 0s 148us/step - loss: 0.5026 - accuracy: 0.7359 - val_

231/231 [==============================] - 0s 145us/step - loss: 0.4810 - accuracy: 0.7619 - val_loss: 0.4993 - val_accuracy: 0.7778
Epoch 168/1000
231/231 [==============================] - 0s 144us/step - loss: 0.4790 - accuracy: 0.7532 - val_loss: 0.4986 - val_accuracy: 0.7778
Epoch 169/1000
231/231 [==============================] - 0s 141us/step - loss: 0.4777 - accuracy: 0.7489 - val_loss: 0.5165 - val_accuracy: 0.7475
Epoch 170/1000
231/231 [==============================] - 0s 137us/step - loss: 0.4808 - accuracy: 0.7576 - val_loss: 0.4953 - val_accuracy: 0.7475
Epoch 171/1000
231/231 [==============================] - 0s 141us/step - loss: 0.4726 - accuracy: 0.7619 - val_loss: 0.5283 - val_accuracy: 0.7374
Epoch 172/1000
231/231 [==============================] - 0s 146us/step - loss: 0.4796 - accuracy: 0.7532 - val_loss: 0.4981 - val_accuracy: 0.7273
Epoch 173/1000
231/231 [==============================] - 0s 148us/step - loss: 0.4811 - accuracy: 0.7706 - val_loss: 0.5051 - 

231/231 [==============================] - 0s 308us/step - loss: 0.4639 - accuracy: 0.7619 - val_loss: 0.4975 - val_accuracy: 0.7778
Epoch 223/1000
231/231 [==============================] - 0s 248us/step - loss: 0.4667 - accuracy: 0.7489 - val_loss: 0.4896 - val_accuracy: 0.7576
Epoch 224/1000
231/231 [==============================] - 0s 238us/step - loss: 0.4619 - accuracy: 0.7576 - val_loss: 0.4977 - val_accuracy: 0.7677
Epoch 225/1000
231/231 [==============================] - 0s 212us/step - loss: 0.4589 - accuracy: 0.7749 - val_loss: 0.4989 - val_accuracy: 0.7576
Epoch 226/1000
231/231 [==============================] - 0s 165us/step - loss: 0.4683 - accuracy: 0.7576 - val_loss: 0.4883 - val_accuracy: 0.7677
Epoch 227/1000
231/231 [==============================] - 0s 167us/step - loss: 0.4637 - accuracy: 0.7619 - val_loss: 0.4993 - val_accuracy: 0.7677
Epoch 228/1000
231/231 [==============================] - 0s 187us/step - loss: 0.4705 - accuracy: 0.7576 - val_loss: 0.5065 - 

...

231/231 [==============================] - 0s 146us/step - loss: 0.3615 - accuracy: 0.8052 - val_loss: 0.4614 - val_accuracy: 0.7677
Epoch 773/1000
231/231 [==============================] - 0s 156us/step - loss: 0.3522 - accuracy: 0.8095 - val_loss: 0.4679 - val_accuracy: 0.7677
Epoch 774/1000
231/231 [==============================] - 0s 142us/step - loss: 0.3577 - accuracy: 0.8009 - val_loss: 0.5059 - val_accuracy: 0.8081
Epoch 775/1000
231/231 [==============================] - 0s 125us/step - loss: 0.3548 - accuracy: 0.8095 - val_loss: 0.4924 - val_accuracy: 0.7576
Epoch 776/1000
231/231 [==============================] - 0s 143us/step - loss: 0.3621 - accuracy: 0.7965 - val_loss: 0.4761 - val_accuracy: 0.7778
Epoch 777/1000
231/231 [==============================] - 0s 144us/step - loss: 0.3640 - accuracy: 0.8009 - val_loss: 0.5527 - val_accuracy: 0.7980
Epoch 778/1000
231/231 [==============================] - 0s 137us/step - loss: 0.3778 - accuracy: 0.7965 - val_loss: 0.5128 - 

231/231 [==============================] - 0s 137us/step - loss: 0.3489 - accuracy: 0.8052 - val_loss: 0.4933 - val_accuracy: 0.7677
Epoch 828/1000
231/231 [==============================] - 0s 148us/step - loss: 0.3526 - accuracy: 0.7965 - val_loss: 0.4798 - val_accuracy: 0.7879
Epoch 829/1000
231/231 [==============================] - 0s 137us/step - loss: 0.3491 - accuracy: 0.8052 - val_loss: 0.4928 - val_accuracy: 0.7778
Epoch 830/1000
231/231 [==============================] - 0s 147us/step - loss: 0.3562 - accuracy: 0.8052 - val_loss: 0.4611 - val_accuracy: 0.7778
Epoch 831/1000
231/231 [==============================] - 0s 143us/step - loss: 0.3588 - accuracy: 0.8095 - val_loss: 0.4686 - val_accuracy: 0.7778
Epoch 832/1000
231/231 [==============================] - 0s 143us/step - loss: 0.3744 - accuracy: 0.7965 - val_loss: 0.4816 - val_accuracy: 0.7778
Epoch 833/1000
231/231 [==============================] - 0s 144us/step - loss: 0.3495 - accuracy: 0.8095 - val_loss: 0.4792 - 

231/231 [==============================] - 0s 140us/step - loss: 0.3665 - accuracy: 0.8009 - val_loss: 0.4781 - val_accuracy: 0.7778
Epoch 883/1000
231/231 [==============================] - 0s 167us/step - loss: 0.3515 - accuracy: 0.7922 - val_loss: 0.4694 - val_accuracy: 0.7778
Epoch 884/1000
231/231 [==============================] - 0s 140us/step - loss: 0.3430 - accuracy: 0.7965 - val_loss: 0.4758 - val_accuracy: 0.7778
Epoch 885/1000
231/231 [==============================] - 0s 136us/step - loss: 0.3637 - accuracy: 0.7922 - val_loss: 0.4687 - val_accuracy: 0.7879
Epoch 886/1000
231/231 [==============================] - 0s 132us/step - loss: 0.3604 - accuracy: 0.8139 - val_loss: 0.4560 - val_accuracy: 0.7879
Epoch 887/1000
231/231 [==============================] - 0s 142us/step - loss: 0.3512 - accuracy: 0.8095 - val_loss: 0.4567 - val_accuracy: 0.7879
Epoch 888/1000
231/231 [==============================] - 0s 135us/step - loss: 0.3467 - accuracy: 0.8052 - val_loss: 0.5114 - 

231/231 [==============================] - 0s 127us/step - loss: 0.3740 - accuracy: 0.8095 - val_loss: 0.4743 - val_accuracy: 0.7778
Epoch 938/1000
231/231 [==============================] - 0s 160us/step - loss: 0.3687 - accuracy: 0.8095 - val_loss: 0.5008 - val_accuracy: 0.7980
Epoch 939/1000
231/231 [==============================] - 0s 146us/step - loss: 0.3479 - accuracy: 0.8009 - val_loss: 0.4580 - val_accuracy: 0.7778
Epoch 940/1000
231/231 [==============================] - 0s 159us/step - loss: 0.3669 - accuracy: 0.8095 - val_loss: 0.5160 - val_accuracy: 0.8081
Epoch 941/1000
231/231 [==============================] - 0s 146us/step - loss: 0.3665 - accuracy: 0.7965 - val_loss: 0.4556 - val_accuracy: 0.7677
Epoch 942/1000
231/231 [==============================] - 0s 145us/step - loss: 0.3471 - accuracy: 0.8095 - val_loss: 0.4508 - val_accuracy: 0.7677
Epoch 943/1000
231/231 [==============================] - 0s 132us/step - loss: 0.3397 - accuracy: 0.8139 - val_loss: 0.4649 - 

231/231 [==============================] - 0s 147us/step - loss: 0.3574 - accuracy: 0.8095 - val_loss: 0.4630 - val_accuracy: 0.7576
Epoch 993/1000
231/231 [==============================] - 0s 156us/step - loss: 0.3486 - accuracy: 0.7965 - val_loss: 0.4631 - val_accuracy: 0.7879
Epoch 994/1000
231/231 [==============================] - 0s 148us/step - loss: 0.3584 - accuracy: 0.8009 - val_loss: 0.4629 - val_accuracy: 0.7879
Epoch 995/1000
231/231 [==============================] - 0s 145us/step - loss: 0.3307 - accuracy: 0.8095 - val_loss: 0.4510 - val_accuracy: 0.7879
Epoch 996/1000
231/231 [==============================] - 0s 137us/step - loss: 0.3404 - accuracy: 0.8052 - val_loss: 0.4503 - val_accuracy: 0.7677
Epoch 997/1000
231/231 [==============================] - 0s 149us/step - loss: 0.3529 - accuracy: 0.7965 - val_loss: 0.5055 - val_accuracy: 0.8283
Epoch 998/1000
231/231 [==============================] - 0s 148us/step - loss: 0.3454 - accuracy: 0.8182 - val_loss: 0.4641 - 

In [162]:
acc_test_over3 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over3*100))

99/99 [==============================] - 0s 99us/step
combination test accuracy: 80.81%


In [163]:
############ Logistic Regression ############

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [164]:
###### logistics
log = LogisticRegression()
log.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [165]:
y_pred = log.predict(X_test)
print('logistic test accuracy %.2f%%' % (log.score(X_test, y_test)*100))

logistic test accuracy 76.62%


In [166]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [167]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 94.92%


In [168]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [169]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 85.86%


In [170]:
############## Random Forest ##############

In [171]:
###### random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [172]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
print('test accuracy: %.2f%%' % (accuracy_score(y_test, y_pred)*100))

test accuracy: 79.22%


In [180]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [181]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 100.00%


In [175]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [176]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 86.87%


In [177]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [182]:
## random forest model with CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv = RandomForestClassifier(n_estimators=100, random_state=123)

accs = cross_val_score(estimator=rfcv, X=X_train, y=y_train, cv=5)
print(accs)
print(accs.mean())

[0.80555556 0.86111111 0.83333333 0.85714286 0.77142857]
0.8257142857142858


In [178]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.96428571 0.96428571 0.85714286 0.92592593 0.88461538]
0.9192511192511192


In [179]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.89361702 0.84782609 0.82608696 0.73913043 0.84782609]
0.8308973172987975
